# Readme

If you would like to run the whole code, you can run it sequentially.<br>
All logs are shown in the code output, and you can check them. <br>
The results of our Best model and baseline are in the section 4.5. <br>
Have fun (*￣︶￣)

# 1 -Data Collection

In [ ]:
# Code to download file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from urllib.request import urlopen
import re
import warnings
warnings.filterwarnings('ignore')
from oauth2client.client import GoogleCredentials

from torch.utils.data import TensorDataset, DataLoader
import torch

from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
import numpy as np

import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.featstruct import remove_variables
from gensim.models import FastText
import gensim.downloader as api
from gensim.models import Word2Vec
from sklearn.preprocessing import LabelEncoder
import spacy 
from nltk.tokenize import TweetTokenizer

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


id = '1dq5pifhJ0-kZLyHXkSgaL1wV5wjZN79h'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('training_data.csv')  

id = '1RC_v8cRj4iXmAbaoJayc5agmo4pnic36'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('validation_data.csv')  

id = '1-yBH-1C4eLufHajEjS06I6McqaCqyKQB'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('testing_data.csv')  


import pandas as pd
training_data = pd.read_csv("/content/training_data.csv")
validation_data = pd.read_csv("/content/validation_data.csv")
testing_data = pd.read_csv("/content/testing_data.csv")

print("------------------------------------")
print("Size of training dataset: {0}".format(len(training_data)))
print("Size of validation dataset: {0}".format(len(validation_data)))
#print("Size of testing dataset: {0}".format(len(testing_data)))
print("------------------------------------")

print("------------------------------------")
print("Sample Data")
print("LABEL: {0} / SENTENCE: {1}".format(validation_data.iloc[-1,0], validation_data.iloc[-1,1]))
print("------------------------------------")


training_original = training_data.copy()
validation_original = validation_data.copy()
testing_original = testing_data.copy()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
------------------------------------
Size of training dataset: 26078
Size of validation dataset: 8705
------------------------------------
------------------------------------
Sample Data
LABEL: i feel for you sniper / SENTENCE: P O O P C
------------------------------------


In [ ]:
## POS and DEP lists:

id = '1gQ3MtX3L6D53s3F99pxpPIAVPcXs411v'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('training_pos.txt')

id = '1_5lg5LiBDkcsftXwQY_suA2liwO8bIGF'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('training_dep.txt')

id = '1NMQQfo215c9LSXkbQwHE8wzqSz55O3es'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('testing_pos.txt')

id = '1S4PQ-cmOAFKCWQ3ierO00jUGenc7TJoG'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('testing_dep.txt')

id = '1uxLMYswQ9b7TA59s_JVt8HnjIuDiD3YG'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('validation_pos.txt')

id = '1hcEpLfmjGxfsFU_T7ZghNuUxR99AaroV'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('validation_dep.txt')

# turning text to lists
def list_from_txt(text_file):
  final_list = []
  for line in open('{}.txt'.format(text_file),'r'):
    line = line.split(" ")
    sub_list = []
    for tag in line:
      sub_list.append(tag.strip())
    final_list.append(sub_list)
  return final_list

# 2 -Preprocessing

In [ ]:
# REMOVING '[SEPA]', additional whitespaces, SEPA in labels
### DO WE DO LOWERCASE??????????

# function to reduce length of consecutive characters: wowwwwwwww -> wowww
def reduce_len(word):
  reduced_word = ''
  for w in word:
    if len(reduced_word) >= 3:
      if w==reduced_word[-3]==reduced_word[-2]==reduced_word[-1]:
        continue
    reduced_word += w
  return reduced_word

def preprocessing_pd(df, sentences_col, labels_col):
  _df = df.copy()
  _df[sentences_col] = _df[sentences_col].str.lower()
  _df[sentences_col] = _df[sentences_col].str.replace(r' +', ' ')
  _df[sentences_col] = _df[sentences_col].str.strip()
  _df[labels_col] = _df[labels_col].str.strip()
  _df[sentences_col] = _df[sentences_col].apply(lambda x: reduce_len(x))
  _df['full_sentences'] = _df[sentences_col]
  _df[sentences_col] = _df[sentences_col].str.split(' ')
  _df[labels_col] = _df[labels_col].str.split(' ')
  return _df

In [ ]:

training_data = preprocessing_pd(training_original,'sents','labels')
validation_data = preprocessing_pd(validation_original,'sents','labels')

In [ ]:
target_y_train = [[y for y in Y.split()] for Y in training_original['labels'].tolist()]
target_y_validation = [[y for y in Y.split()] for Y in validation_original['labels'].tolist()]

# 3 -Embedding

- ⚠️ IMPORTANT Pt.1⚠️ FUNCTION TO CREATE MARKINGS FOR SENTENCES 👀 
Arguments:
1. training dataframe (train_dat)
2. marking type: 'pos', 'dep', 'tag' (type)

given these three arguments...
1. returns the list of sentences with markings instead of words <br>
(returned with index 0) <br>
e.g. data_marking(*args*) [ 0 ] = [ [ NOUN , PRN , VERB ] , [ ... ] ]
<br>
<br>
2. returns the dictionary of a unique mark type to a number <br>
(returned with index 1) <br>
e.g. data_marking(*args*) [ 1 ] = { NOUN : 1 , PRN : 2 , VERB : 3 , ... }

## 3.1. Helper Functions

In [ ]:
# fixing tokenisation issues with fixed sequences
def remerge_sent(sent):
  i = 0
  while i < len(sent)-1:
      to = sent[i]
      if not to.whitespace_:
          nto = sent[i+1]
          sent.merge(to.idx, nto.idx+len(nto))
      i += 1
  return sent

In [ ]:
# generate syntactic version of each sentence 
### train_dat : {training dataframe}
### type : {'pos', 'dep', 'tag'}

def data_marking(train_dat, type):
  
  # HELPER FUNCTION 2
  # function to remerge tokens as spaCy splits can increase token counts e.g. cant -> ca nt
  # retains only top most marking of split tokens
  def remerge_sent(sent):
    i = 0
    while i < len(sent)-1:
        to = sent[i]
        if not to.whitespace_:
            nto = sent[i+1]
            sent.merge(to.idx, nto.idx+len(nto))
        i += 1
    return sent

  # creating marked representation of each sentence for a designated type
  mark_list = []
  i = 0
  for row in train_dat.itertuples():
    temp_list = []
    sent = remerge_sent(nlp(row.full_sentences))
    if len(sent)!=len(row.sents):
      sent = remerge_sent(sent)
    i += 1
    print(i)
    if type=='pos':
      for token in sent:
        temp_list.append(token.pos_)

    if type=='dep':
      for token in sent:
        temp_list.append(token.dep_)

    if type=='tag':
      for token in sent:
        temp_list.append(token.tag_)

    mark_list.append(temp_list)

  return mark_list

- ⚠️ IMPORTANT Pt.2⚠️ (more helper functions) <br>
 SOME FUNCTIONS <br>
 1. **word_list** = list of all words in sentences <br>
 2. **word_index_generate** = list of indices of words in word list <br>
 3. **spelling_corrector** = tries to fix spelling if a word in the test set is OOV (looks at similar words with edit-distance = 1 in training/validation set and then changes to that word). <br>
> for example: test set contains OOV word 'ccheese', training set contains word 'cheese' which is 1 edit-distance away from 'ccheese' ( remove 'c' ) so 'ccheese' is converted to 'cheese' in the test set. <br>

 4. **preprocessing_testing** = preprocess test set using spelling_correcter (which also requires a list of training/val words) <br>

In [ ]:
## functions to generate:
## word_list and word_index, as well as preprocessing testing data (since we need to use word list)

def word_list_generate(sentences):
    word_list = set()
    word_list.add('[PAD]')
    word_list.add('[UNKNOWN]')
    for sent in sentences:
        for word in sent:
            word_list.add(word)
    word_list = list(word_list)
    return word_list

# function for generating word index
def word_index_generate(word_list):
    word_index = {}
    i = 0
    for w in word_list:
        word_index[w] = i
        i += 1
    return word_index

def spelling_corrector(train_val_word_list, test_data):
    corrections_dic = {}
    for i, sentence in enumerate(test_data):
      for j, test_word in enumerate(sentence):
        if len(test_word) > 3 and test_word not in train_val_word_list and test_word not in corrections_dic:
          for train_word in train_val_word_list:
            if nltk.edit_distance(test_word, train_word, substitution_cost=2, transpositions=True)==1:
              corrections_dic[test_word] = train_word
    for n, sentence in enumerate(test_data):
      for m, word in enumerate(sentence):
        if word in corrections_dic:
          test_data[n][m]=test_data[n][m].replace(word,corrections_dic[word])
    return test_data

def preprocessing_testing(df, sentences_col, train_val_word_list):
  _df = df.copy()
  _df[sentences_col] = _df[sentences_col].str.lower()
  _df[sentences_col] = _df[sentences_col].str.replace(r' +', ' ')
  _df[sentences_col] = _df[sentences_col].str.strip()
  _df['full_sentences'] = _df['sents']
  _df[sentences_col] = _df[sentences_col].apply(lambda x: reduce_len(x))
  _df[sentences_col] = _df[sentences_col].str.split(' ')
  spelling_corrector(train_val_word_list, _df[sentences_col]) ### [spelling corrector]
  return _df


In [ ]:
# preprocessing test set
testing_data = testing_original.copy()
train_val_sentences = pd.concat([training_data['sents'], validation_data['sents']])
train_val_word_list = word_list_generate(train_val_sentences)
testing_preprocessed = preprocessing_testing(testing_data, 'sents', train_val_word_list)
testing_data = testing_preprocessed
x_testing = testing_data['sents'].tolist()

In [ ]:
# example use-case of spelling_corrector
print('original sentence:', testing_original['sents'][93])
print('spelling corrected:', testing_data['sents'][93])
print('STUPIUD -> stupid')

original sentence: STUPIUD PAUSE
spelling corrected: ['stupid', 'pause']
STUPIUD -> stupid


In [ ]:
def dict_ix(mark_list):
  mark_dic = {}
  for sentence in mark_list:
    for mark in sentence:
      if mark not in mark_dic:
        mark_dic[mark] = len(mark_dic)
  return mark_dic

## 3.2. Syntactic *Embedding*

### POS tag

In [ ]:
training_pos_marking = list_from_txt('training_pos')
validation_pos_marking = list_from_txt('validation_pos')
testing_pos_marking = list_from_txt('testing_pos')

In [ ]:
pos_ix = dict_ix(training_pos_marking + validation_pos_marking + testing_pos_marking)
pos_embedding = np.eye(len(list(pos_ix.values())))

### Dep tag

In [ ]:
training_dep_marking = list_from_txt('training_dep')
validation_dep_marking = list_from_txt('validation_dep')
testing_dep_marking = list_from_txt('testing_dep')

In [ ]:
dep_ix = dict_ix(training_dep_marking + validation_dep_marking + testing_dep_marking)
dep_embedding = np.eye(len(list(dep_ix.values())))

## 3.3. Semantic Embedding

In [ ]:
#glove_twitter_50 = api.load("glove-twitter-50")
ft_train_val = FastText(sentences=pd.concat([validation_data['sents'], training_data['sents']]),window=3,size=100, min_count=1)

## 3.4. Domain Embedding

In [ ]:
##### DOTA WIKI ######
char_url = "https://dota2.fandom.com/wiki/Heroes"
char_page = urlopen(char_url)
char_html_bytes = char_page.read()
char_html = char_html_bytes.decode("utf-8")
result = re.findall('black;">(.*?)</span></div></div><div style=', char_html) + ['Wraith_King','Zeus','Weaver']
char_list = []
for char in result:
  if ' ' in char:
    char = char.replace(' ','_')
  if "'" in char:
    char = char.replace("'",'%27')
  char_list.append(char)

# PREPROCESSING HTML PAGE TO GET RELEVANT INFORMATION

import nltk
from nltk.corpus import stopwords

def preprocess_html(html_):
  sww = stopwords.words('english')
  html = re.sub(r'<.+?>', '', html_)
  html = re.sub(r"\n|\t|\'s|\'",'',html)
  html = re.sub(r'^.*?(Gameplay\[\]Playstyle)','',html)
  html = re.sub("\d+", "",html)
  html = re.sub(r'\S*\[\]','',html)
  html = re.sub("-",'',html)
  html = re.sub("[^\w\s]",' ',html)
  html = re.sub("NewPP.*$",'',html)
  html = html.strip()
  html = re.sub(" +",' ',html).lower()
  html = html.split(' ')
  return html

# LIST OF CHAMP=SPECIFIC INFORMATION
dota_domain_docs_sww = []
dota_domain_docs_nosww = []
for char in char_list:
  url = "https://dota2.fandom.com/wiki/{}/Guide".format(char)
  page = urlopen(url)
  html_bytes = page.read()
  html = html_bytes.decode("utf-8")
  dota_domain_docs_sww.append(preprocess_html(html))


from gensim.models import FastText
ft_DOTAWIKI = FastText(sentences=dota_domain_docs_sww, window=10, size=50, min_count=1)

In [ ]:
##### CONDA TEST ######
id = '1G_0175o6cznxpRSsbPSL73FVWchauMKs'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('CONDA_test_comments.csv') 
CONDA_domain = pd.read_csv('/content/CONDA_test_comments.csv')
CONDA_domain['utterance'] = CONDA_domain['utterance'].str.lower()
CONDA_domain['utterance'] = CONDA_domain['utterance'].str.replace(r' +', ' ')
CONDA_domain['utterance'] = CONDA_domain['utterance'].str.strip()
CONDA_domain['utterance'] = CONDA_domain['utterance'].apply(lambda x: reduce_len(x))
CONDA_domain['utterance'] = CONDA_domain['utterance'].str.split(' ')

from gensim.models import FastText
ft_CONDA = FastText(sentences=CONDA_domain, window=3, size=100, min_count=1)

## 3.5. Generate word_to_ix and tag_to_ix

In [ ]:
full_sentences = pd.concat([training_data['sents'], validation_data['sents'], testing_data['sents']])
full_word_list = word_list_generate(full_sentences)
full_word_to_ix = word_index_generate(full_word_list) 
print('sent_word_list:',full_word_list[:20])
print()
len(full_word_list)
len(full_word_to_ix)

sent_word_list: ['yea', 'divided', 'saboid', 'fckng', 'sentry', 'caught', 'drawing', 'comenden', 'tower', 'gagos', 'yeahhh', 'gamme', 'scouting', 'blyadi', 'okey', 'rim', 'morphling', 'diferent', 'nobody', 'nxt']



10963

In [ ]:
# tag to index
START_TAG = "<START>"
STOP_TAG = "<STOP>"
tag_to_ix = {START_TAG:0, STOP_TAG:1}
for tags in target_y_train + target_y_validation:
    for tag in tags:
        if tag not in tag_to_ix:
            tag_to_ix[tag] = len(tag_to_ix)
tag_to_ix

{'<START>': 0,
 '<STOP>': 1,
 'C': 8,
 'D': 7,
 'O': 2,
 'P': 4,
 'S': 6,
 'SEPA': 5,
 'T': 3}

In [ ]:
#### convert dataset into idxs

x_train = training_data['sents'].tolist()
x_validation = validation_data['sents'].tolist()
x_testing = testing_data['sents'].tolist()


def index_generate(sentences, ind):
    ind_list = []
    for sent in sentences:
        ind_list.append([ind[w] for w in sent])
    return ind_list

train_input_index =  index_generate(x_train,full_word_to_ix)
train_output_index = index_generate(target_y_train,tag_to_ix)
val_input_index = index_generate(x_validation, full_word_to_ix)
val_output_index = index_generate(target_y_validation,tag_to_ix)
train_dep_index = index_generate(training_dep_marking,dep_ix)
train_pos_index =  index_generate(training_pos_marking,pos_ix)
val_dep_index = index_generate(validation_dep_marking,dep_ix)
val_pos_index =  index_generate(validation_pos_marking,pos_ix)
test_input_index = index_generate(x_testing,full_word_to_ix)
test_dep_index = index_generate(testing_dep_marking,dep_ix)
test_pos_index =  index_generate(testing_pos_marking,pos_ix)

## 3.6 Generate Embedding Matrix

In [ ]:
WORD_EMBEDDING_DIM_1 = 50 ## DOTA WIKI
WORD_EMBEDDING_DIM_2 = 100 ## TRAIN / VAL
WORD_EMBEDDING_DIM_3 = 100 ## CONDA 
WV_EMBEDDING_DIM = WORD_EMBEDDING_DIM_1 + WORD_EMBEDDING_DIM_2 + WORD_EMBEDDING_DIM_3
embedding_matrix = []
for word in full_word_list:
    try:
      a = ft_DOTAWIKI.wv[word]
    except:
      a = [0]*WORD_EMBEDDING_DIM_1
    try:
      b = ft_train_val.wv[word]
    except:
      b = [0]*WORD_EMBEDDING_DIM_2
    try:
      c = ft_CONDA.wv[word]
    except:
      c = [0]*WORD_EMBEDDING_DIM_3

    embedding_matrix.append(np.concatenate([a,b,c]))
embedding_matrix = np.array(embedding_matrix)
embedding_matrix.shape

embedding_dim = len(pos_ix) + len(dep_ix) + WV_EMBEDDING_DIM

In [ ]:
embedding_dim

312

In [ ]:
# DOTA WIKI ONLY
WORD_EMBEDDING_DIM_1 = 50 ## DOTA WIKI
WORD_EMBEDDING_DIM_2 = 100 ## TRAIN / VAL
WV_EMBEDDING_DIM_domain_experiment = WORD_EMBEDDING_DIM_1 + WORD_EMBEDDING_DIM_2
embedding_matrix_domain_experiment = []
for word in full_word_list:
    try:
      a = ft_DOTAWIKI.wv[word]
    except:
      a = [0]*WORD_EMBEDDING_DIM_1
    try:
      b = ft_train_val.wv[word]
    except:
      b = [0]*WORD_EMBEDDING_DIM_2


    embedding_matrix_domain_experiment.append(np.concatenate([a,b]))
embedding_matrix_domain_experiment = np.array(embedding_matrix_domain_experiment)
print(embedding_matrix_domain_experiment.shape)

embedding_dim_domain_experiment = len(pos_ix) + len(dep_ix) + WV_EMBEDDING_DIM_domain_experiment

# 4- Ablation Experiment

## 4.1. Ablation Study 1: Different input embedding

### Model 1

Embedding: Fasttext(training+validation data) + pos + dep + fasttext(dota-wiki + CONDA test)

In [ ]:

torch.manual_seed(1)

def argmax(vec):
    # return the argmax as a python int
    _, idx = torch.max(vec, 1)
    return idx.item()


# Compute log sum exp in a numerically stable way for the forward algorithm
def log_sum_exp(vec):
    max_score = vec[0, argmax(vec)]
    max_score_broadcast = max_score.view(1, -1).expand(1, vec.size()[1])
    return max_score + \
        torch.log(torch.sum(torch.exp(vec - max_score_broadcast)))

class BiLSTM_CRF(nn.Module):

    def __init__(self, vocab_size, tag_to_ix, embedding_dim, hidden_dim, wv_embedding_dim, method):
        super(BiLSTM_CRF, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.tag_to_ix = tag_to_ix
        self.tagset_size = len(tag_to_ix)
        self.wv_embedding_dim = wv_embedding_dim
        self.method = method
        self.word_embeds = nn.Embedding(vocab_size, wv_embedding_dim)
        self.word_embeds.weight.data.copy_(torch.from_numpy(embedding_matrix))
        
        self.pos_embeds = nn.Embedding(pos_embedding.shape[0], pos_embedding.shape[0])
        self.pos_embeds.weight.data.copy_(torch.from_numpy(pos_embedding))
        
        self.dep_embeds = nn.Embedding(dep_embedding.shape[0], dep_embedding.shape[0])
        self.dep_embeds.weight.data.copy_(torch.from_numpy(dep_embedding))
        
        self.lstm = nn.LSTM(embedding_dim, hidden_dim // 2,
                            num_layers=2, bidirectional=True,dropout=0.5)

        # Maps the output of the LSTM into tag space.
        self.hidden2tag = nn.Linear(hidden_dim*2, self.tagset_size)

        # Matrix of transition parameters.  Entry i,j is the score of
        # transitioning *to* i *from* j.
        self.transitions = nn.Parameter(
            torch.randn(self.tagset_size, self.tagset_size))

        # These two statements enforce the constraint that we never transfer
        # to the start tag and we never transfer from the stop tag
        self.transitions.data[tag_to_ix[START_TAG], :] = -10000
        self.transitions.data[:, tag_to_ix[STOP_TAG]] = -10000

        self.hidden = self.hidden_initial()
        self.dropout_lstm=nn.Dropout(p=0.5)
        
    def hidden_initial(self):
        return (torch.randn(4, 1, self.hidden_dim // 2).to(device), ### different layers 需要改
                torch.randn(4, 1, self.hidden_dim // 2).to(device)) ###

    def _forward_alg(self, feats):
        # Do the forward algorithm to compute the partition function
        init_alphas = torch.full((1, self.tagset_size), -10000.).to(device)
        # START_TAG has all of the score.
        init_alphas[0][self.tag_to_ix[START_TAG]] = 0.
        #print('_forward_alg')
        # Wrap in a variable so that we will get automatic backprop
        forward_var = init_alphas

        # Iterate through the sentence
        for feat in feats:
            alphas_t = []  # The forward tensors at this timestep
            for next_tag in range(self.tagset_size):
                # broadcast the emission score: it is the same regardless of
                # the previous tag
                emit_score = feat[next_tag].view(
                    1, -1).expand(1, self.tagset_size)
                # the ith entry of trans_score is the score of transitioning to
                # next_tag from i
                trans_score = self.transitions[next_tag].view(1, -1)
                # The ith entry of next_tag_var is the value for the
                # edge (i -> next_tag) before we do log-sum-exp
                next_tag_var = forward_var + trans_score + emit_score
                # The forward variable for this tag is log-sum-exp of all the
                # scores.
                alphas_t.append(log_sum_exp(next_tag_var).view(1))
            forward_var = torch.cat(alphas_t).view(1, -1)
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        alpha = log_sum_exp(terminal_var)
        return alpha
    # method = dot
    def attention(self,lstm_out,hidden_out, hidden_dim, method):    
        h_out_T = torch.transpose(hidden_out,1,2)
        h_out_T0 = h_out_T   
        h_out_0 = hidden_out   
        for i in range(lstm_out.size()[0]-1):
          h_out_T = torch.cat((h_out_T, h_out_T0), 0)
          hidden_out = torch.cat((hidden_out, h_out_0), 0)
        W = nn.Linear(hidden_dim,hidden_dim,bias=False)
        if method == 'dot':
            attention_w = F.softmax(torch.bmm(lstm_out, h_out_T),dim=-1)
        elif method == 'scaled-dot':
            attention_w = F.softmax(torch.bmm(lstm_out, h_out_T)/np.sqrt(self.hidden_dim),dim=-1)
        elif method == 'cosine':
            attention_w = F.softmax(torch.bmm(lstm_out,h_out_T)/(torch.norm(lstm_out)*torch.norm(h_out_T)),dim=-1)

        elif method == 'general':
            attention_w = F.softmax(torch.bmm(lstm_out, W(h_out_T), h_out_T),dim=-1)

        attention_output = torch.bmm(attention_w, hidden_out)
        concat_output = torch.cat((attention_output, lstm_out), 1)
        return concat_output

    def lstm_ft_get(self, sentence, pos, dep, method):
        self.hidden = self.hidden_initial()
        embeds = torch.cat((self.word_embeds(sentence).view(len(sentence), 1, -1),self.pos_embeds(pos).view(len(pos), 1, -1),self.dep_embeds(dep).view(len(dep), 1, -1)),2)
        lstm_out, self.hidden = self.lstm(embeds, self.hidden)
        hidden_out = torch.cat((self.hidden[0].view(2, 2, 1, -1)[1,0,:,:],self.hidden[0].view(2, 2, 1, -1)[1,1,:,:]),1) ###################不同layer这里需要改
        hidden_out = hidden_out.unsqueeze(0)
        attention_out = self.attention(lstm_out,hidden_out,hidden_dim,method)
        attention_out = attention_out.view(-1,self.hidden_dim*2)
        attention_out = self.dropout_lstm(attention_out)
        lstm_fts = self.hidden2tag(attention_out)
        return lstm_fts

    def sentence_scoring(self, feats, tags):
        # Gives the score of a provided tag sequence
        score = torch.zeros(1).to(device)
        tags = torch.cat([torch.tensor([self.tag_to_ix[START_TAG]], dtype=torch.long).to(device), tags])
        for i, feat in enumerate(feats):
            score = score + self.transitions[tags[i + 1], tags[i]] + feat[tags[i + 1]]
        score = score + self.transitions[self.tag_to_ix[STOP_TAG], tags[-1]]
        return score

    def viterbi_decoding(self, feats):
        backpointers = []
        # Initialize the viterbi variables in log space
        init_vvars = torch.full((1, self.tagset_size), -10000.).to(device)
        init_vvars[0][self.tag_to_ix[START_TAG]] = 0
        # forward_var at step i holds the viterbi variables for step i-1
        forward_var = init_vvars
        for feat in feats:
            bptrs_t = []  # holds the backpointers for this step
            viterbivars_t = []  # holds the viterbi variables for this step

            for next_tag in range(self.tagset_size):
                # next_tag_var[i] holds the viterbi variable for tag i at the
                # previous step, plus the score of transitioning
                # from tag i to next_tag.
                # We don't include the emission scores here because the max
                # does not depend on them (we add them in below)
                next_tag_var = forward_var + self.transitions[next_tag]
                best_tag_id = argmax(next_tag_var)
                bptrs_t.append(best_tag_id)
                viterbivars_t.append(next_tag_var[0][best_tag_id].view(1))
            # Now add in the emission scores, and assign forward_var to the set
            # of viterbi variables we just computed
            forward_var = (torch.cat(viterbivars_t) + feat).view(1, -1)
            backpointers.append(bptrs_t)

        # Transition to STOP_TAG
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        best_tag_id = argmax(terminal_var)
        path_score = terminal_var[0][best_tag_id]

        # Follow the back pointers to decode the best path.
        best_path = [best_tag_id]
        for bptrs_t in reversed(backpointers):
            best_tag_id = bptrs_t[best_tag_id]
            best_path.append(best_tag_id)
        # Pop off the start tag (we dont want to return that to the caller)
        start = best_path.pop()
        assert start == self.tag_to_ix[START_TAG]  # Sanity check
        best_path.reverse()
        return path_score, best_path

    def neg_ll(self, sentence, tags, pos, dep, method):
        feats = self.lstm_ft_get(sentence, pos, dep, method)
        forward_score = self._forward_alg(feats)
        gold_score = self.sentence_scoring(feats, tags)
        return forward_score - gold_score

    def forward(self, sentence, pos, dep):
        lstm_fts = self.lstm_ft_get(sentence, pos, dep, method)
        score, tag_seq = self.viterbi_decoding(lstm_fts)
        return score, tag_seq



def accuracy_calculation(model, input_index, output_index, pos_index, dep_index):
  c, total = 0, 0
  pred_list, truth = [], []

  for i in range(len(input_index)):
    sentence_in = torch.tensor(input_index[i], dtype=torch.long).to(device)
    pos = torch.tensor(pos_index[i], dtype=torch.long).to(device)
    dep = torch.tensor(dep_index[i], dtype=torch.long).to(device)
    _, pred = model.forward(sentence_in, pos, dep)
    for j in range(len(output_index[i])):
      total += 1
      
      pred_list.append(pred[j])
      truth.append(output_index[i][j])

      if pred[j] == output_index[i][j]:
        c += 1

  accuracy = c/total

  return truth, pred_list, accuracy


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
hidden_dim = 250

model = BiLSTM_CRF(len(full_word_to_ix), tag_to_ix, embedding_dim, hidden_dim, WV_EMBEDDING_DIM, method='scaled-dot').to(device)
optimizer = optim.SGD(model.parameters(), lr=0.1, weight_decay=1e-4)

import datetime
for epoch in range(2):  
    time1 = datetime.datetime.now()
    train_loss = 0
    method='scaled-dot'
    model.train()
    for i in range(len(train_input_index)):
        #print(i)
        tags_index = train_output_index[i]
        dep_index = train_dep_index[i]
        pos_index = train_pos_index[i]
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is,
        # turn them into Tensors of word indices.
        sentence_in = torch.tensor(train_input_index[i], dtype=torch.long).to(device)
        pos = torch.tensor(pos_index, dtype=torch.long).to(device)
        dep = torch.tensor(dep_index, dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)
        # Step 3. Run our forward pass.
        loss = model.neg_ll(sentence_in, targets, pos, dep, method='scaled-dot')
        # Step 4. Compute the loss, gradients, and update the parameters by
        # calling optimizer.step()
        loss.backward()
        optimizer.step()

        train_loss+=loss.item()

    model.eval()
     # 
    _, _, train_acc = accuracy_calculation(model,train_input_index,train_output_index,train_pos_index,train_dep_index)
    _, _, val_acc = accuracy_calculation(model,val_input_index,val_output_index,val_pos_index,val_dep_index)

    val_loss = 0
    for i in range(len(val_input_index)):
        tags_index = val_output_index[i]
        pos_index = val_pos_index[i]
        dep_index = val_dep_index[i]
        sentence_in = torch.tensor(val_input_index[i], dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)
        pos = torch.tensor(pos_index, dtype=torch.long).to(device)
        dep = torch.tensor(dep_index, dtype=torch.long).to(device)
        loss = model.neg_ll(sentence_in, targets, pos, dep, method='scaled-dot')
        val_loss+=loss.item()
    time2 = datetime.datetime.now()

    print("Epoch:%d, Training loss: %.2f, train acc: %.4f, val loss: %.2f, val acc: %.4f, time: %.2fs" %(epoch+1, train_loss,train_acc, val_loss, val_acc, (time2-time1).total_seconds()))

y_pred, y_true, _ = accuracy_calculation(model,val_input_index,val_output_index,val_pos_index,val_dep_index)
def decode_output(output_list):
    ix_to_tag = {v:k for k,v in tag_to_ix.items()}
    return [ix_to_tag[output] for output in output_list]
y_true_decode = decode_output(y_true)
y_pred_decode = decode_output(y_pred)
from sklearn.metrics import classification_report
print(classification_report(y_true_decode,y_pred_decode,digits=4))

Epoch:1, Training loss: 34709.38, train acc: 0.9865, val loss: 2175.60, val acc: 0.9844, time: 630.77s
Epoch:2, Training loss: 5511.21, train acc: 0.9958, val loss: 1519.26, val acc: 0.9933, time: 619.77s
              precision    recall  f1-score   support

           C     0.9677    0.9802    0.9739      1620
           D     0.9045    0.9863    0.9436       365
           O     0.9997    0.9921    0.9959     19130
           P     0.9970    0.9997    0.9983      3925
           S     0.9831    0.9973    0.9901      3275
        SEPA     1.0000    1.0000    1.0000      3603
           T     0.9598    0.9819    0.9707      1436

    accuracy                         0.9933     33354
   macro avg     0.9731    0.9911    0.9818     33354
weighted avg     0.9935    0.9933    0.9933     33354



In [ ]:
import sklearn
from sklearn.metrics import f1_score
sklearn.metrics.f1_score(y_true_decode, y_pred_decode,average='micro')

0.9932841638184325

### Model 2

Embedding: Fasttext(training+validation data) +  fasttext(dota-wiki + CONDA test)  <br>

In [ ]:
torch.manual_seed(1)
embedding_dim = 250

def argmax(vec):
    # return the argmax as a python int
    _, idx = torch.max(vec, 1)
    return idx.item()


# Compute log sum exp in a numerically stable way for the forward algorithm
def log_sum_exp(vec):
    max_score = vec[0, argmax(vec)]
    max_score_broadcast = max_score.view(1, -1).expand(1, vec.size()[1])
    return max_score + \
        torch.log(torch.sum(torch.exp(vec - max_score_broadcast)))

class BiLSTM_CRF(nn.Module):

    def __init__(self, vocab_size, tag_to_ix, embedding_dim, hidden_dim, wv_embedding_dim, method):
        super(BiLSTM_CRF, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.tag_to_ix = tag_to_ix
        self.tagset_size = len(tag_to_ix)
        self.wv_embedding_dim = wv_embedding_dim
        self.method = method
        self.word_embeds = nn.Embedding(vocab_size, wv_embedding_dim)
        self.word_embeds.weight.data.copy_(torch.from_numpy(embedding_matrix))
        
        self.lstm = nn.LSTM(embedding_dim, hidden_dim // 2,
                            num_layers=2, bidirectional=True,dropout=0.5)

        # Maps the output of the LSTM into tag space.
        self.hidden2tag = nn.Linear(hidden_dim*2, self.tagset_size)

        # Matrix of transition parameters.  Entry i,j is the score of
        # transitioning *to* i *from* j.
        self.transitions = nn.Parameter(
            torch.randn(self.tagset_size, self.tagset_size))

        # These two statements enforce the constraint that we never transfer
        # to the start tag and we never transfer from the stop tag
        self.transitions.data[tag_to_ix[START_TAG], :] = -10000
        self.transitions.data[:, tag_to_ix[STOP_TAG]] = -10000

        self.hidden = self.hidden_initial()
        self.dropout_lstm=nn.Dropout(p=0.5)
        
    def hidden_initial(self):
        return (torch.randn(4, 1, self.hidden_dim // 2).to(device), ### different layers 需要改
                torch.randn(4, 1, self.hidden_dim // 2).to(device)) ###

    def _forward_alg(self, feats):
        # Do the forward algorithm to compute the partition function
        init_alphas = torch.full((1, self.tagset_size), -10000.).to(device)
        # START_TAG has all of the score.
        init_alphas[0][self.tag_to_ix[START_TAG]] = 0.
        #print('_forward_alg')
        # Wrap in a variable so that we will get automatic backprop
        forward_var = init_alphas

        # Iterate through the sentence
        for feat in feats:
            alphas_t = []  # The forward tensors at this timestep
            for next_tag in range(self.tagset_size):
                # broadcast the emission score: it is the same regardless of
                # the previous tag
                emit_score = feat[next_tag].view(
                    1, -1).expand(1, self.tagset_size)
                # the ith entry of trans_score is the score of transitioning to
                # next_tag from i
                trans_score = self.transitions[next_tag].view(1, -1)
                # The ith entry of next_tag_var is the value for the
                # edge (i -> next_tag) before we do log-sum-exp
                next_tag_var = forward_var + trans_score + emit_score
                # The forward variable for this tag is log-sum-exp of all the
                # scores.
                alphas_t.append(log_sum_exp(next_tag_var).view(1))
            forward_var = torch.cat(alphas_t).view(1, -1)
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        alpha = log_sum_exp(terminal_var)
        return alpha
    # method = dot
    def attention(self,lstm_out,hidden_out, hidden_dim, method):
        h_out_T = torch.transpose(hidden_out,1,2)
        
        h_out_T_1 = h_out_T   
        h_out_0 = hidden_out   
        for i in range(lstm_out.size()[0]-1):
          h_out_T = torch.cat((h_out_T, h_out_T_1), 0)
          hidden_out = torch.cat((hidden_out, h_out_0), 0)
        W = nn.Linear(hidden_dim,hidden_dim,bias=False)
        if method == 'dot':
            attention_w = F.softmax(torch.bmm(lstm_out, h_out_T),dim=-1)
        elif method == 'scaled-dot':
            attention_w = F.softmax(torch.bmm(lstm_out, h_out_T)/np.sqrt(self.hidden_dim),dim=-1)
        elif method == 'cosine':
            attention_w = F.softmax(torch.bmm(lstm_out,h_out_T)/(torch.norm(lstm_out)*torch.norm(h_out_T)),dim=-1)

        elif method == 'general':
            attention_w = F.softmax(torch.bmm(lstm_out, W(h_out_T), h_out_T),dim=-1)

        attention_output = torch.bmm(attention_w, hidden_out)
        concat_output = torch.cat((attention_output, lstm_out), 1)
        return concat_output

    def lstm_ft_get(self, sentence, method):
        self.hidden = self.hidden_initial()

        w_embeds = self.word_embeds(sentence).view(len(sentence), 1, -1)
        lstm_out, self.hidden = self.lstm(w_embeds, self.hidden)

        hidden_out = torch.cat((self.hidden[0].view(2, 2, 1, -1)[1,0,:,:],self.hidden[0].view(2, 2, 1, -1)[1,1,:,:]),1) ###################不同layer这里需要改
        hidden_out = hidden_out.unsqueeze(0)
        attention_out = self.attention(lstm_out,hidden_out,hidden_dim,method)
        attention_out = attention_out.view(-1,self.hidden_dim*2)
        attention_out = self.dropout_lstm(attention_out)
        lstm_fts = self.hidden2tag(attention_out)
        return lstm_fts

    def sentence_scoring(self, feats, tags):
        # Gives the score of a provided tag sequence
        score = torch.zeros(1).to(device)
        tags = torch.cat([torch.tensor([self.tag_to_ix[START_TAG]], dtype=torch.long).to(device), tags])
        for i, feat in enumerate(feats):
            score = score + \
                self.transitions[tags[i + 1], tags[i]] + feat[tags[i + 1]]
        score = score + self.transitions[self.tag_to_ix[STOP_TAG], tags[-1]]
        return score

    def viterbi_decoding(self, feats):
        backpointers = []

        # Initialize the viterbi variables in log space
        init_vvars = torch.full((1, self.tagset_size), -10000.).to(device)
        init_vvars[0][self.tag_to_ix[START_TAG]] = 0

        # forward_var at step i holds the viterbi variables for step i-1
        forward_var = init_vvars
        for feat in feats:
            bptrs_t = []  # holds the backpointers for this step
            viterbivars_t = []  # holds the viterbi variables for this step

            for next_tag in range(self.tagset_size):
                # next_tag_var[i] holds the viterbi variable for tag i at the
                # previous step, plus the score of transitioning
                # from tag i to next_tag.
                # We don't include the emission scores here because the max
                # does not depend on them (we add them in below)
                next_tag_var = forward_var + self.transitions[next_tag]
                best_tag_id = argmax(next_tag_var)
                bptrs_t.append(best_tag_id)
                viterbivars_t.append(next_tag_var[0][best_tag_id].view(1))
            # Now add in the emission scores, and assign forward_var to the set
            # of viterbi variables we just computed
            forward_var = (torch.cat(viterbivars_t) + feat).view(1, -1)
            backpointers.append(bptrs_t)

        # Transition to STOP_TAG
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        best_tag_id = argmax(terminal_var)
        path_score = terminal_var[0][best_tag_id]

        # Follow the back pointers to decode the best path.
        best_path = [best_tag_id]
        for bptrs_t in reversed(backpointers):
            best_tag_id = bptrs_t[best_tag_id]
            best_path.append(best_tag_id)
        # Pop off the start tag (we dont want to return that to the caller)
        start = best_path.pop()
        assert start == self.tag_to_ix[START_TAG]  # Sanity check
        best_path.reverse()
        return path_score, best_path

    def neg_ll(self, sentence, tags, method):
        feats = self.lstm_ft_get(sentence, method)
        #print('neg_ll')
        forward_score = self._forward_alg(feats)
        gold_score = self.sentence_scoring(feats, tags)
        return forward_score - gold_score

    #def forward(self, sentence, ent, pos, dep, bert):  # dont confuse this with _forward_alg above.
    def forward(self, sentence):
        #print('forward')
        # Get the emission scores from the BiLSTM

        lstm_fts = self.lstm_ft_get(sentence, method)
        # Find the best path, given the features.
        score, tag_seq = self.viterbi_decoding(lstm_fts)
        return score, tag_seq



def accuracy_calculation(model, input_index, output_index):

  c, total = 0, 0

  pred_list, truth = [], []

  #print('accuracy_calculation')
  for i in range(len(input_index)):
    sentence_in = torch.tensor(input_index[i], dtype=torch.long).to(device)

    _, pred = model.forward(sentence_in)
    for j in range(len(output_index[i])):
      total += 1
      
      pred_list.append(pred[j])
      truth.append(output_index[i][j])

      if pred[j] == output_index[i][j]:
        c += 1

  accuracy = c/total

  return truth, pred_list, accuracy


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
hidden_dim = 250

model = BiLSTM_CRF(len(full_word_to_ix), tag_to_ix, embedding_dim, hidden_dim, WV_EMBEDDING_DIM, method='scaled-dot').to(device)
optimizer = optim.SGD(model.parameters(), lr=0.1, weight_decay=1e-4)

import datetime
for epoch in range(2):  
    time1 = datetime.datetime.now()
    train_loss = 0
    method='scaled-dot'
    model.train()
    for i in range(len(train_input_index)):
        #print(i)
        tags_index = train_output_index[i]
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is,
        # turn them into Tensors of word indices.
        sentence_in = torch.tensor(train_input_index[i], dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)
        # Step 3. Run our forward pass.
        loss = model.neg_ll(sentence_in, targets, method='scaled-dot')
        # Step 4. Compute the loss, gradients, and update the parameters by
        # calling optimizer.step()
        loss.backward()
        optimizer.step()

        train_loss+=loss.item()

    model.eval()
     # 
    _, _, train_acc = accuracy_calculation(model,train_input_index,train_output_index)
    _, _, val_acc = accuracy_calculation(model,val_input_index,val_output_index)

    val_loss = 0
    for i in range(len(val_input_index)):
        tags_index = val_output_index[i]
        pos_index = val_pos_index[i]
        dep_index = val_dep_index[i]
        sentence_in = torch.tensor(val_input_index[i], dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)
        pos = torch.tensor(pos_index, dtype=torch.long).to(device)
        dep = torch.tensor(dep_index, dtype=torch.long).to(device)
        loss = model.neg_ll(sentence_in, targets, method='scaled-dot')
        val_loss+=loss.item()
    time2 = datetime.datetime.now()

    print("Epoch:%d, Training loss: %.2f, train acc: %.4f, val loss: %.2f, val acc: %.4f, time: %.2fs" %(epoch+1, train_loss,train_acc, val_loss, val_acc, (time2-time1).total_seconds()))

y_pred, y_true, _ = accuracy_calculation(model,val_input_index,val_output_index)
def decode_output(output_list):
    ix_to_tag = {v:k for k,v in tag_to_ix.items()}
    return [ix_to_tag[output] for output in output_list]
y_true_decode = decode_output(y_true)
y_pred_decode = decode_output(y_pred)
from sklearn.metrics import classification_report
print(classification_report(y_true_decode,y_pred_decode,digits=4))


Epoch:1, Training loss: 37623.89, train acc: 0.9850, val loss: 2187.91, val acc: 0.9835, time: 632.68s
Epoch:2, Training loss: 5299.76, train acc: 0.9959, val loss: 1394.35, val acc: 0.9923, time: 634.36s
              precision    recall  f1-score   support

           C     0.9586    0.9868    0.9725      1594
           D     0.9322    0.9322    0.9322       398
           O     0.9992    0.9915    0.9954     19132
           P     0.9962    0.9995    0.9978      3923
           S     0.9801    0.9985    0.9892      3261
        SEPA     1.0000    1.0000    1.0000      3603
           T     0.9585    0.9757    0.9670      1443

    accuracy                         0.9924     33354
   macro avg     0.9750    0.9835    0.9792     33354
weighted avg     0.9926    0.9924    0.9925     33354



In [ ]:
import sklearn
from sklearn.metrics import f1_score
sklearn.metrics.f1_score(y_true_decode, y_pred_decode,average='micro')

0.9924446842957366

### Model 3

Embedding: Fasttext(training+validation data) + pos + dep + fasttext(dota-wiki)

In [ ]:
torch.manual_seed(1)

def argmax(vec):
    # return the argmax as a python int
    _, idx = torch.max(vec, 1)
    return idx.item()


# Compute log sum exp in a numerically stable way for the forward algorithm
def log_sum_exp(vec):
    max_score = vec[0, argmax(vec)]
    max_score_broadcast = max_score.view(1, -1).expand(1, vec.size()[1])
    return max_score + \
        torch.log(torch.sum(torch.exp(vec - max_score_broadcast)))

class BiLSTM_CRF(nn.Module):

    def __init__(self, vocab_size, tag_to_ix, embedding_dim, hidden_dim, wv_embedding_dim, method):
        super(BiLSTM_CRF, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.tag_to_ix = tag_to_ix
        self.tagset_size = len(tag_to_ix)
        self.wv_embedding_dim = wv_embedding_dim
        self.method = method
        self.word_embeds = nn.Embedding(vocab_size, wv_embedding_dim)
        self.word_embeds.weight.data.copy_(torch.from_numpy(embedding_matrix_domain_experiment))
        self.pos_embeds = nn.Embedding(pos_embedding.shape[0], pos_embedding.shape[0])
        self.pos_embeds.weight.data.copy_(torch.from_numpy(pos_embedding))
        self.dep_embeds = nn.Embedding(dep_embedding.shape[0], dep_embedding.shape[0])
        self.dep_embeds.weight.data.copy_(torch.from_numpy(dep_embedding))
        
        self.lstm = nn.LSTM(embedding_dim, hidden_dim // 2,
                            num_layers=2, bidirectional=True,dropout=0.5)

        # Maps the output of the LSTM into tag space.
        self.hidden2tag = nn.Linear(hidden_dim*2, self.tagset_size)

        # Matrix of transition parameters.  Entry i,j is the score of
        # transitioning *to* i *from* j.
        self.transitions = nn.Parameter(
            torch.randn(self.tagset_size, self.tagset_size))

        # These two statements enforce the constraint that we never transfer
        # to the start tag and we never transfer from the stop tag
        self.transitions.data[tag_to_ix[START_TAG], :] = -10000
        self.transitions.data[:, tag_to_ix[STOP_TAG]] = -10000

        self.hidden = self.hidden_initial()
        self.dropout_lstm=nn.Dropout(p=0.5)
        
    def hidden_initial(self):
        return (torch.randn(4, 1, self.hidden_dim // 2).to(device), ### different layers 需要改
                torch.randn(4, 1, self.hidden_dim // 2).to(device)) ###

    def _forward_alg(self, feats):
        # Do the forward algorithm to compute the partition function
        init_alphas = torch.full((1, self.tagset_size), -10000.).to(device)
        # START_TAG has all of the score.
        init_alphas[0][self.tag_to_ix[START_TAG]] = 0.
        #print('_forward_alg')
        # Wrap in a variable so that we will get automatic backprop
        forward_var = init_alphas

        # Iterate through the sentence
        for feat in feats:
            alphas_t = []  # The forward tensors at this timestep
            for next_tag in range(self.tagset_size):
                # broadcast the emission score: it is the same regardless of
                # the previous tag
                emit_score = feat[next_tag].view(
                    1, -1).expand(1, self.tagset_size)
                # the ith entry of trans_score is the score of transitioning to
                # next_tag from i
                trans_score = self.transitions[next_tag].view(1, -1)
                # The ith entry of next_tag_var is the value for the
                # edge (i -> next_tag) before we do log-sum-exp
                next_tag_var = forward_var + trans_score + emit_score
                # The forward variable for this tag is log-sum-exp of all the
                # scores.
                alphas_t.append(log_sum_exp(next_tag_var).view(1))
            forward_var = torch.cat(alphas_t).view(1, -1)
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        alpha = log_sum_exp(terminal_var)
        return alpha
    def attention(self,lstm_out,hidden_out, hidden_dim, method):
        h_out_T = torch.transpose(hidden_out,1,2)
        h_out_T_1 = h_out_T   
        h_out_0 = hidden_out   
        for i in range(lstm_out.size()[0]-1):
          h_out_T = torch.cat((h_out_T, h_out_T_1), 0)
          hidden_out = torch.cat((hidden_out, h_out_0), 0)
        W = nn.Linear(hidden_dim,hidden_dim,bias=False)
        if method == 'dot':
            attention_w = F.softmax(torch.bmm(lstm_out, h_out_T),dim=-1)
        elif method == 'scaled-dot':
            attention_w = F.softmax(torch.bmm(lstm_out, h_out_T)/np.sqrt(self.hidden_dim),dim=-1)
        elif method == 'cosine':
            attention_w = F.softmax(torch.bmm(lstm_out,h_out_T)/(torch.norm(lstm_out)*torch.norm(h_out_T)),dim=-1)

        elif method == 'general':
            attention_w = F.softmax(torch.bmm(lstm_out, W(h_out_T), h_out_T),dim=-1)

        attention_output = torch.bmm(attention_w, hidden_out)
        concat_output = torch.cat((attention_output, lstm_out), 1)
        return concat_output

    def lstm_ft_get(self, sentence, pos, dep, method):
        self.hidden = self.hidden_initial()
        embeds = torch.cat((self.word_embeds(sentence).view(len(sentence), 1, -1),self.pos_embeds(pos).view(len(pos), 1, -1),self.dep_embeds(dep).view(len(dep), 1, -1)),2)
        lstm_out, self.hidden = self.lstm(embeds, self.hidden)
        hidden_out = torch.cat((self.hidden[0].view(2, 2, 1, -1)[1,0,:,:],self.hidden[0].view(2, 2, 1, -1)[1,1,:,:]),1) ###################不同layer这里需要改
        hidden_out = hidden_out.unsqueeze(0)
        attention_out = self.attention(lstm_out,hidden_out,hidden_dim,method)
        attention_out = attention_out.view(-1,self.hidden_dim*2)
        attention_out = self.dropout_lstm(attention_out)
        lstm_fts = self.hidden2tag(attention_out)
        return lstm_fts

    def sentence_scoring(self, feats, tags):
        score = torch.zeros(1).to(device)
        tags = torch.cat([torch.tensor([self.tag_to_ix[START_TAG]], dtype=torch.long).to(device), tags])
        for i, feat in enumerate(feats):
            score = score + self.transitions[tags[i + 1], tags[i]] + feat[tags[i + 1]]
        score = score + self.transitions[self.tag_to_ix[STOP_TAG], tags[-1]]
        return score

    def negative_ll(self, sentence, tags, pos, dep, method):
        feats = self.lstm_ft_get(sentence, pos, dep, method)
        forward_score = self._forward_alg(feats)
        gold_score = self.sentence_scoring(feats, tags)
        return forward_score - gold_score

    def viterbi_decoding(self, feats):
        backpointers = []

        # Initialize the viterbi variables in log space
        init_vvars = torch.full((1, self.tagset_size), -10000.).to(device)
        init_vvars[0][self.tag_to_ix[START_TAG]] = 0

        # forward_var at step i holds the viterbi variables for step i-1
        forward_var = init_vvars
        for feat in feats:
            bptrs_t = []  # holds the backpointers for this step
            viterbivars_t = []  # holds the viterbi variables for this step

            for next_tag in range(self.tagset_size):
                # next_tag_var[i] holds the viterbi variable for tag i at the
                # previous step, plus the score of transitioning
                # from tag i to next_tag.
                # We don't include the emission scores here because the max
                # does not depend on them (we add them in below)
                next_tag_var = forward_var + self.transitions[next_tag]
                best_tag_id = argmax(next_tag_var)
                bptrs_t.append(best_tag_id)
                viterbivars_t.append(next_tag_var[0][best_tag_id].view(1))
            # Now add in the emission scores, and assign forward_var to the set
            # of viterbi variables we just computed
            forward_var = (torch.cat(viterbivars_t) + feat).view(1, -1)
            backpointers.append(bptrs_t)

        # Transition to STOP_TAG
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        best_tag_id = argmax(terminal_var)
        path_score = terminal_var[0][best_tag_id]

        # Follow the back pointers to decode the best path.
        best_path = [best_tag_id]
        for bptrs_t in reversed(backpointers):
            best_tag_id = bptrs_t[best_tag_id]
            best_path.append(best_tag_id)
        # Pop off the start tag (we dont want to return that to the caller)
        start = best_path.pop()
        assert start == self.tag_to_ix[START_TAG]  # Sanity check
        best_path.reverse()
        return path_score, best_path

    def forward(self, sentence, pos, dep):
        #print('forward')
        # Get the emission scores from the BiLSTM

        lstm_fts = self.lstm_ft_get(sentence, pos, dep, method)
        # Find the best path, given the features.
        score, tag_seq = self.viterbi_decoding(lstm_fts)
        return score, tag_seq



def accuracy_calculation(model, input_index, output_index, pos_index, dep_index):
  c, total = 0, 0

  pred_list, truth = [], []

  #print('accuracy_calculation')
  for i in range(len(input_index)):
    sentence_in = torch.tensor(input_index[i], dtype=torch.long).to(device)
    pos = torch.tensor(pos_index[i], dtype=torch.long).to(device)
    dep = torch.tensor(dep_index[i], dtype=torch.long).to(device)


    _, pred = model.forward(sentence_in, pos, dep)
    for j in range(len(output_index[i])):
      total += 1
      
      pred_list.append(pred[j])
      truth.append(output_index[i][j])

      if pred[j] == output_index[i][j]:
        c += 1

  accuracy = c/total

  return truth, pred_list, accuracy

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
hidden_dim = 250

model = BiLSTM_CRF(len(full_word_to_ix), tag_to_ix, embedding_dim_domain_experiment, hidden_dim, WV_EMBEDDING_DIM_domain_experiment, method='scaled-dot').to(device)
optimizer = optim.SGD(model.parameters(), lr=0.1, weight_decay=1e-4)

import datetime
for epoch in range(2):  
    time1 = datetime.datetime.now()
    train_loss = 0
    method='scaled-dot'
    model.train()
    for i in range(len(train_input_index)):
        #print(i)
        tags_index = train_output_index[i]
        dep_index = train_dep_index[i]
        pos_index = train_pos_index[i]
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is,
        # turn them into Tensors of word indices.
        sentence_in = torch.tensor(train_input_index[i], dtype=torch.long).to(device)
        pos = torch.tensor(pos_index, dtype=torch.long).to(device)
        dep = torch.tensor(dep_index, dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)
        # Step 3. Run our forward pass.
        loss = model.neg_ll(sentence_in, targets, pos, dep, method='scaled-dot')
        # Step 4. Compute the loss, gradients, and update the parameters by
        # calling optimizer.step()
        loss.backward()
        optimizer.step()

        train_loss+=loss.item()

    model.eval()
     # 
    _, _, train_acc = accuracy_calculation(model,train_input_index,train_output_index,train_pos_index,train_dep_index)
    _, _, val_acc = accuracy_calculation(model,val_input_index,val_output_index,val_pos_index,val_dep_index)

    val_loss = 0
    for i in range(len(val_input_index)):
        tags_index = val_output_index[i]
        pos_index = val_pos_index[i]
        dep_index = val_dep_index[i]
        sentence_in = torch.tensor(val_input_index[i], dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)
        pos = torch.tensor(pos_index, dtype=torch.long).to(device)
        dep = torch.tensor(dep_index, dtype=torch.long).to(device)
        loss = model.neg_ll(sentence_in, targets, pos, dep, method='scaled-dot')
        val_loss+=loss.item()
    time2 = datetime.datetime.now()

    print("Epoch:%d, Training loss: %.2f, train acc: %.4f, val loss: %.2f, val acc: %.4f, time: %.2fs" %(epoch+1, train_loss,train_acc, val_loss, val_acc, (time2-time1).total_seconds()))

y_pred, y_true, _ = accuracy_calculation(model,val_input_index,val_output_index,val_pos_index,val_dep_index)
def decode_output(output_list):
    ix_to_tag = {v:k for k,v in tag_to_ix.items()}
    return [ix_to_tag[output] for output in output_list]
y_true_decode = decode_output(y_true)
y_pred_decode = decode_output(y_pred)
from sklearn.metrics import classification_report
print(classification_report(y_true_decode,y_pred_decode,digits=4))

Epoch:1, Training loss: 33116.77, train acc: 0.9830, val loss: 2355.28, val acc: 0.9812, time: 651.40s
Epoch:2, Training loss: 6398.47, train acc: 0.9935, val loss: 1433.08, val acc: 0.9909, time: 646.57s
              precision    recall  f1-score   support

           C     0.9592    0.9746    0.9668      1615
           D     0.8065    0.9497    0.8723       338
           O     0.9987    0.9908    0.9947     19138
           P     0.9929    0.9997    0.9963      3909
           S     0.9895    0.9958    0.9926      3301
        SEPA     1.0000    1.0000    1.0000      3603
           T     0.9455    0.9579    0.9517      1450

    accuracy                         0.9907     33354
   macro avg     0.9560    0.9812    0.9678     33354
weighted avg     0.9911    0.9907    0.9908     33354


In [ ]:
import sklearn
from sklearn.metrics import f1_score
sklearn.metrics.f1_score(y_true_decode, y_pred_decode,average='micro')

0.9907147692882463


## 4.2. Ablation Study 2: Different attention strategy

### Model 1

Calculation method is "scaled dot-product" and position is in the second layer

In [ ]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import math
from numpy.linalg import norm

torch.manual_seed(1)

def argmax(vec):
    # return the argmax as a python int
    _, idx = torch.max(vec, 1)
    return idx.item()


# Compute log sum exp in a numerically stable way for the forward algorithm
def log_sum_exp(vec):
    max_score = vec[0, argmax(vec)]
    max_score_broadcast = max_score.view(1, -1).expand(1, vec.size()[1])
    return max_score + \
        torch.log(torch.sum(torch.exp(vec - max_score_broadcast)))

class BiLSTM_CRF(nn.Module):

    def __init__(self, vocab_size, tag_to_ix, embedding_dim, hidden_dim, wv_embedding_dim, method):
        super(BiLSTM_CRF, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.tag_to_ix = tag_to_ix
        self.tagset_size = len(tag_to_ix)
        self.wv_embedding_dim = wv_embedding_dim
        self.method = method
        self.word_embeds = nn.Embedding(vocab_size, wv_embedding_dim)
        self.word_embeds.weight.data.copy_(torch.from_numpy(embedding_matrix))
        
        self.pos_embeds = nn.Embedding(pos_embedding.shape[0], pos_embedding.shape[0])
        self.pos_embeds.weight.data.copy_(torch.from_numpy(pos_embedding))
        
        self.dep_embeds = nn.Embedding(dep_embedding.shape[0], dep_embedding.shape[0])
        self.dep_embeds.weight.data.copy_(torch.from_numpy(dep_embedding))
        
        self.lstm = nn.LSTM(embedding_dim, hidden_dim // 2,
                            num_layers=2, bidirectional=True,dropout=0.5)

        # Maps the output of the LSTM into tag space.
        self.hidden2tag = nn.Linear(hidden_dim*2, self.tagset_size)

        # Matrix of transition parameters.  Entry i,j is the score of
        # transitioning *to* i *from* j.
        self.transitions = nn.Parameter(
            torch.randn(self.tagset_size, self.tagset_size))

        # These two statements enforce the constraint that we never transfer
        # to the start tag and we never transfer from the stop tag
        self.transitions.data[tag_to_ix[START_TAG], :] = -10000
        self.transitions.data[:, tag_to_ix[STOP_TAG]] = -10000

        self.hidden = self.hidden_initial()
        self.dropout_lstm=nn.Dropout(p=0.5)
        
    def hidden_initial(self):
        return (torch.randn(4, 1, self.hidden_dim // 2).to(device), ### different layers 需要改
                torch.randn(4, 1, self.hidden_dim // 2).to(device)) ###

    def _forward_alg(self, feats):
        # Do the forward algorithm to compute the partition function
        init_alphas = torch.full((1, self.tagset_size), -10000.).to(device)
        # START_TAG has all of the score.
        init_alphas[0][self.tag_to_ix[START_TAG]] = 0.
        #print('_forward_alg')
        # Wrap in a variable so that we will get automatic backprop
        forward_var = init_alphas

        # Iterate through the sentence
        for feat in feats:
            alphas_t = []  # The forward tensors at this timestep
            for next_tag in range(self.tagset_size):
                # broadcast the emission score: it is the same regardless of
                # the previous tag
                emit_score = feat[next_tag].view(
                    1, -1).expand(1, self.tagset_size)
                # the ith entry of trans_score is the score of transitioning to
                # next_tag from i
                trans_score = self.transitions[next_tag].view(1, -1)
                # The ith entry of next_tag_var is the value for the
                # edge (i -> next_tag) before we do log-sum-exp
                next_tag_var = forward_var + trans_score + emit_score
                # The forward variable for this tag is log-sum-exp of all the
                # scores.
                alphas_t.append(log_sum_exp(next_tag_var).view(1))
            forward_var = torch.cat(alphas_t).view(1, -1)
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        alpha = log_sum_exp(terminal_var)
        return alpha
    # method = dot
    def attention(self,lstm_out,hidden_out, hidden_dim, method):

        h_out_T = torch.transpose(hidden_out,1,2)
        
        h_out_T_1 = h_out_T   
        h_out_0 = hidden_out   
        for i in range(lstm_out.size()[0]-1):
          h_out_T = torch.cat((h_out_T, h_out_T_1), 0)
          hidden_out = torch.cat((hidden_out, h_out_0), 0)
        W = nn.Linear(hidden_dim,hidden_dim,bias=False)
        if method == 'dot':
            attention_w = F.softmax(torch.bmm(lstm_out, h_out_T),dim=-1)
        elif method == 'scaled-dot':
            attention_w = F.softmax(torch.bmm(lstm_out, h_out_T)/np.sqrt(self.hidden_dim),dim=-1)
        elif method == 'cosine':
            attention_w = F.softmax(torch.bmm(lstm_out,h_out_T)/(torch.norm(lstm_out)*torch.norm(h_out_T)),dim=-1)

        elif method == 'general':
            attention_w = F.softmax(torch.bmm(lstm_out, W(h_out_T), h_out_T),dim=-1)

        attention_output = torch.bmm(attention_w, hidden_out)
        concat_output = torch.cat((attention_output, lstm_out), 1)
        return concat_output

    def lstm_ft_get(self, sentence, pos, dep, method):
        self.hidden = self.hidden_initial()
        #print('lstm_ft_get')

        embeds = torch.cat((self.word_embeds(sentence).view(len(sentence), 1, -1),self.pos_embeds(pos).view(len(pos), 1, -1),self.dep_embeds(dep).view(len(dep), 1, -1)),2)
        lstm_out, self.hidden = self.lstm(embeds, self.hidden)
        #print('lstm_out.shape',lstm_out.shape)
        hidden_out = torch.cat((self.hidden[0].view(2, 2, 1, -1)[1,0,:,:],self.hidden[0].view(2, 2, 1, -1)[1,1,:,:]),1) ###################不同layer这里需要改
        hidden_out = hidden_out.unsqueeze(0)
        #print('hidden_out.shape',hidden_out.shape)
        attention_out = self.attention(lstm_out,hidden_out,hidden_dim,method)
        attention_out = attention_out.view(-1,self.hidden_dim*2)
        #print('attention_out.shape',attention_out.shape)
        attention_out = self.dropout_lstm(attention_out)
        lstm_fts = self.hidden2tag(attention_out)
        #print('lstm_fts.shape',lstm_fts.shape)
        return lstm_fts

    def sentence_scoring(self, feats, tags):
        # Gives the score of a provided tag sequence
        score = torch.zeros(1).to(device)
        tags = torch.cat([torch.tensor([self.tag_to_ix[START_TAG]], dtype=torch.long).to(device), tags])
        for i, feat in enumerate(feats):
            score = score + \
                self.transitions[tags[i + 1], tags[i]] + feat[tags[i + 1]]
        score = score + self.transitions[self.tag_to_ix[STOP_TAG], tags[-1]]
        return score

    def viterbi_decoding(self, feats):
        backpointers = []

        # Initialize the viterbi variables in log space
        init_vvars = torch.full((1, self.tagset_size), -10000.).to(device)
        init_vvars[0][self.tag_to_ix[START_TAG]] = 0

        # forward_var at step i holds the viterbi variables for step i-1
        forward_var = init_vvars
        for feat in feats:
            bptrs_t = []  # holds the backpointers for this step
            viterbivars_t = []  # holds the viterbi variables for this step

            for next_tag in range(self.tagset_size):
                # next_tag_var[i] holds the viterbi variable for tag i at the
                # previous step, plus the score of transitioning
                # from tag i to next_tag.
                # We don't include the emission scores here because the max
                # does not depend on them (we add them in below)
                next_tag_var = forward_var + self.transitions[next_tag]
                best_tag_id = argmax(next_tag_var)
                bptrs_t.append(best_tag_id)
                viterbivars_t.append(next_tag_var[0][best_tag_id].view(1))
            # Now add in the emission scores, and assign forward_var to the set
            # of viterbi variables we just computed
            forward_var = (torch.cat(viterbivars_t) + feat).view(1, -1)
            backpointers.append(bptrs_t)

        # Transition to STOP_TAG
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        best_tag_id = argmax(terminal_var)
        path_score = terminal_var[0][best_tag_id]

        # Follow the back pointers to decode the best path.
        best_path = [best_tag_id]
        for bptrs_t in reversed(backpointers):
            best_tag_id = bptrs_t[best_tag_id]
            best_path.append(best_tag_id)
        # Pop off the start tag (we dont want to return that to the caller)
        start = best_path.pop()
        assert start == self.tag_to_ix[START_TAG]  # Sanity check
        best_path.reverse()
        return path_score, best_path

    def neg_ll(self, sentence, tags, pos, dep, method):
        feats = self.lstm_ft_get(sentence, pos, dep, method)
        forward_score = self._forward_alg(feats)
        gold_score = self.sentence_scoring(feats, tags)
        return forward_score - gold_score

    #def forward(self, sentence, ent, pos, dep, bert):  # dont confuse this with _forward_alg above.
    def forward(self, sentence, pos, dep):
        #print('forward')
        # Get the emission scores from the BiLSTM
        lstm_fts = self.lstm_ft_get(sentence, pos, dep, method)
        # Find the best path, given the features.
        score, tag_seq = self.viterbi_decoding(lstm_fts)
        return score, tag_seq



def accuracy_calculation(model, input_index, output_index, pos_index, dep_index):

  c, total = 0, 0

  pred_list, truth = [], []

  #print('accuracy_calculation')
  for i in range(len(input_index)):
    sentence_in = torch.tensor(input_index[i], dtype=torch.long).to(device)
    pos = torch.tensor(pos_index[i], dtype=torch.long).to(device)
    dep = torch.tensor(dep_index[i], dtype=torch.long).to(device)


    _, pred = model.forward(sentence_in, pos, dep)
    for j in range(len(output_index[i])):
      total += 1
      
      pred_list.append(pred[j])
      truth.append(output_index[i][j])

      if pred[j] == output_index[i][j]:
        c += 1

  accuracy = c/total

  return truth, pred_list, accuracy


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
hidden_dim = 250

model = BiLSTM_CRF(len(full_word_to_ix), tag_to_ix, embedding_dim, hidden_dim, WV_EMBEDDING_DIM, method='scaled-dot').to(device)
optimizer = optim.SGD(model.parameters(), lr=0.1, weight_decay=1e-4)

import datetime
for epoch in range(2):  
    time1 = datetime.datetime.now()
    train_loss = 0
    method='scaled-dot'
    model.train()
    for i in range(len(train_input_index)):
        #print(i)
        tags_index = train_output_index[i]
        dep_index = train_dep_index[i]
        pos_index = train_pos_index[i]
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is,
        # turn them into Tensors of word indices.
        sentence_in = torch.tensor(train_input_index[i], dtype=torch.long).to(device)
        pos = torch.tensor(pos_index, dtype=torch.long).to(device)
        dep = torch.tensor(dep_index, dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)
        # Step 3. Run our forward pass.
        loss = model.neg_ll(sentence_in, targets, pos, dep, method='scaled-dot')
        # Step 4. Compute the loss, gradients, and update the parameters by
        # calling optimizer.step()
        loss.backward()
        optimizer.step()

        train_loss+=loss.item()

    model.eval()
     # 
    _, _, train_acc = accuracy_calculation(model,train_input_index,train_output_index,train_pos_index,train_dep_index)
    _, _, val_acc = accuracy_calculation(model,val_input_index,val_output_index,val_pos_index,val_dep_index)

    val_loss = 0
    for i in range(len(val_input_index)):
        tags_index = val_output_index[i]
        pos_index = val_pos_index[i]
        dep_index = val_dep_index[i]
        sentence_in = torch.tensor(val_input_index[i], dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)
        pos = torch.tensor(pos_index, dtype=torch.long).to(device)
        dep = torch.tensor(dep_index, dtype=torch.long).to(device)
        loss = model.neg_ll(sentence_in, targets, pos, dep, method='scaled-dot')
        val_loss+=loss.item()
    time2 = datetime.datetime.now()

    print("Epoch:%d, Training loss: %.2f, train acc: %.4f, val loss: %.2f, val acc: %.4f, time: %.2fs" %(epoch+1, train_loss,train_acc, val_loss, val_acc, (time2-time1).total_seconds()))

y_pred, y_true, _ = accuracy_calculation(model,val_input_index,val_output_index,val_pos_index,val_dep_index)
def decode_output(output_list):
    ix_to_tag = {v:k for k,v in tag_to_ix.items()}
    return [ix_to_tag[output] for output in output_list]
y_true_decode = decode_output(y_true)
y_pred_decode = decode_output(y_pred)
from sklearn.metrics import classification_report
print(classification_report(y_true_decode,y_pred_decode,digits=4))


Epoch:1, Training loss: 35176.46, train acc: 0.9870, val loss: 2071.47, val acc: 0.9855, time: 638.67s
Epoch:2, Training loss: 5214.27, train acc: 0.9962, val loss: 1309.13, val acc: 0.9926, time: 669.32s
              precision    recall  f1-score   support

           C     0.9714    0.9708    0.9711      1642
           D     0.8719    0.9666    0.9168       359
           O     0.9983    0.9928    0.9955     19089
           P     0.9982    0.9990    0.9986      3933
           S     0.9868    0.9957    0.9912      3292
        SEPA     1.0000    1.0000    1.0000      3603
           T     0.9551    0.9770    0.9659      1436

    accuracy                         0.9926     33354
   macro avg     0.9688    0.9860    0.9770     33354
weighted avg     0.9928    0.9926    0.9926     33354



In [ ]:
import sklearn
from sklearn.metrics import f1_score
sklearn.metrics.f1_score(y_true_decode, y_pred_decode,average='micro')

0.9925346285303112

### Model 2

Calculation method is "dot-product" and position is in the second layer


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
hidden_dim = 250

model = BiLSTM_CRF(len(full_word_to_ix), tag_to_ix, embedding_dim, hidden_dim, WV_EMBEDDING_DIM, method='dot').to(device)
optimizer = optim.SGD(model.parameters(), lr=0.1, weight_decay=1e-4)

import datetime
for epoch in range(2):  
    time1 = datetime.datetime.now()
    train_loss = 0
    method='dot'
    model.train()
    for i in range(len(train_input_index)):
        #print(i)
        tags_index = train_output_index[i]
        dep_index = train_dep_index[i]
        pos_index = train_pos_index[i]
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is,
        # turn them into Tensors of word indices.
        sentence_in = torch.tensor(train_input_index[i], dtype=torch.long).to(device)
        pos = torch.tensor(pos_index, dtype=torch.long).to(device)
        dep = torch.tensor(dep_index, dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)
        # Step 3. Run our forward pass.
        loss = model.neg_ll(sentence_in, targets, pos, dep, method='dot')
        # Step 4. Compute the loss, gradients, and update the parameters by
        # calling optimizer.step()
        loss.backward()
        optimizer.step()

        train_loss+=loss.item()

    model.eval()
     # 
    _, _, train_acc = accuracy_calculation(model,train_input_index,train_output_index,train_pos_index,train_dep_index)
    _, _, val_acc = accuracy_calculation(model,val_input_index,val_output_index,val_pos_index,val_dep_index)

    val_loss = 0
    for i in range(len(val_input_index)):
        tags_index = val_output_index[i]
        pos_index = val_pos_index[i]
        dep_index = val_dep_index[i]
        sentence_in = torch.tensor(val_input_index[i], dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)
        pos = torch.tensor(pos_index, dtype=torch.long).to(device)
        dep = torch.tensor(dep_index, dtype=torch.long).to(device)
        loss = model.neg_ll(sentence_in, targets, pos, dep, method='dot')
        val_loss+=loss.item()
    time2 = datetime.datetime.now()

    print("Epoch:%d, Training loss: %.2f, train acc: %.4f, val loss: %.2f, val acc: %.4f, time: %.2fs" %(epoch+1, train_loss,train_acc, val_loss, val_acc, (time2-time1).total_seconds()))

y_pred, y_true, _ = accuracy_calculation(model,val_input_index,val_output_index,val_pos_index,val_dep_index)
def decode_output(output_list):
    ix_to_tag = {v:k for k,v in tag_to_ix.items()}
    return [ix_to_tag[output] for output in output_list]
y_true_decode = decode_output(y_true)
y_pred_decode = decode_output(y_pred)
from sklearn.metrics import classification_report
print(classification_report(y_true_decode,y_pred_decode,digits=4))

Epoch:1, Training loss: 34068.97, train acc: 0.9849, val loss: 2308.79, val acc: 0.9830, time: 642.52s
Epoch:2, Training loss: 5765.05, train acc: 0.9944, val loss: 1524.71, val acc: 0.9918, time: 662.61s
              precision    recall  f1-score   support

           C     0.9720    0.9667    0.9693      1650
           D     0.8216    0.9970    0.9008       328
           O     0.9995    0.9912    0.9954     19144
           P     0.9967    0.9997    0.9982      3924
           S     0.9837    0.9973    0.9905      3277
        SEPA     1.0000    1.0000    1.0000      3603
           T     0.9517    0.9790    0.9651      1428

    accuracy                         0.9921     33354
   macro avg     0.9607    0.9901    0.9742     33354
weighted avg     0.9925    0.9921    0.9922     33354



In [ ]:
import sklearn
from sklearn.metrics import f1_score
sklearn.metrics.f1_score(y_true_decode, y_pred_decode,average='micro')

0.9920849073574384

### Model 3

Calculation method is "cosine" and position is in the second layer

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
hidden_dim = 250

model = BiLSTM_CRF(len(full_word_to_ix), tag_to_ix, embedding_dim, hidden_dim, WV_EMBEDDING_DIM, method='cosine').to(device)
optimizer = optim.SGD(model.parameters(), lr=0.1, weight_decay=1e-4)

import datetime
for epoch in range(2):  
    time1 = datetime.datetime.now()
    train_loss = 0
    method='cosine'
    model.train()
    for i in range(len(train_input_index)):
        #print(i)
        tags_index = train_output_index[i]
        dep_index = train_dep_index[i]
        pos_index = train_pos_index[i]
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is,
        # turn them into Tensors of word indices.
        sentence_in = torch.tensor(train_input_index[i], dtype=torch.long).to(device)
        pos = torch.tensor(pos_index, dtype=torch.long).to(device)
        dep = torch.tensor(dep_index, dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)
        # Step 3. Run our forward pass.
        loss = model.neg_ll(sentence_in, targets, pos, dep, method='cosine')
        # Step 4. Compute the loss, gradients, and update the parameters by
        # calling optimizer.step()
        loss.backward()
        optimizer.step()

        train_loss+=loss.item()

    model.eval()
     # 
    _, _, train_acc = accuracy_calculation(model,train_input_index,train_output_index,train_pos_index,train_dep_index)
    _, _, val_acc = accuracy_calculation(model,val_input_index,val_output_index,val_pos_index,val_dep_index)

    val_loss = 0
    for i in range(len(val_input_index)):
        tags_index = val_output_index[i]
        pos_index = val_pos_index[i]
        dep_index = val_dep_index[i]
        sentence_in = torch.tensor(val_input_index[i], dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)
        pos = torch.tensor(pos_index, dtype=torch.long).to(device)
        dep = torch.tensor(dep_index, dtype=torch.long).to(device)
        loss = model.neg_ll(sentence_in, targets, pos, dep, method='cosine')
        val_loss+=loss.item()
    time2 = datetime.datetime.now()

    print("Epoch:%d, Training loss: %.2f, train acc: %.4f, val loss: %.2f, val acc: %.4f, time: %.2fs" %(epoch+1, train_loss,train_acc, val_loss, val_acc, (time2-time1).total_seconds()))

y_pred, y_true, _ = accuracy_calculation(model,val_input_index,val_output_index,val_pos_index,val_dep_index)
def decode_output(output_list):
    ix_to_tag = {v:k for k,v in tag_to_ix.items()}
    return [ix_to_tag[output] for output in output_list]
y_true_decode = decode_output(y_true)
y_pred_decode = decode_output(y_pred)
from sklearn.metrics import classification_report
print(classification_report(y_true_decode,y_pred_decode,digits=4))

Epoch:1, Training loss: 31311.03, train acc: 0.9830, val loss: 2355.21, val acc: 0.9817, time: 659.33s
Epoch:2, Training loss: 6202.13, train acc: 0.9936, val loss: 1506.62, val acc: 0.9906, time: 646.72s
              precision    recall  f1-score   support

           C     0.9537    0.9684    0.9610      1616
           D     0.8492    0.9854    0.9123       343
           O     0.9989    0.9915    0.9952     19127
           P     0.9959    0.9995    0.9977      3922
           S     0.9786    0.9979    0.9881      3258
        SEPA     1.0000    0.9994    0.9997      3605
           T     0.9483    0.9393    0.9438      1483

    accuracy                         0.9904     33354
   macro avg     0.9607    0.9831    0.9711     33354
weighted avg     0.9907    0.9904    0.9905     33354



In [ ]:
import sklearn
from sklearn.metrics import f1_score
sklearn.metrics.f1_score(y_true_decode, y_pred_decode,average='micro')

0.9904359297235714

### Model 4

Calculation method is "scaled dot-product" and position is in the first layer

In [ ]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import math
from numpy.linalg import norm

torch.manual_seed(1)

def argmax(vec):
    # return the argmax as a python int
    _, idx = torch.max(vec, 1)
    return idx.item()


# Compute log sum exp in a numerically stable way for the forward algorithm
def log_sum_exp(vec):
    max_score = vec[0, argmax(vec)]
    max_score_broadcast = max_score.view(1, -1).expand(1, vec.size()[1])
    return max_score + \
        torch.log(torch.sum(torch.exp(vec - max_score_broadcast)))

class BiLSTM_CRF(nn.Module):

    def __init__(self, vocab_size, tag_to_ix, embedding_dim, hidden_dim, wv_embedding_dim, method):
        super(BiLSTM_CRF, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.tag_to_ix = tag_to_ix
        self.tagset_size = len(tag_to_ix)
        self.wv_embedding_dim = wv_embedding_dim
        self.method = method
        self.word_embeds = nn.Embedding(vocab_size, wv_embedding_dim)
        self.word_embeds.weight.data.copy_(torch.from_numpy(embedding_matrix))
        
        self.pos_embeds = nn.Embedding(pos_embedding.shape[0], pos_embedding.shape[0])
        self.pos_embeds.weight.data.copy_(torch.from_numpy(pos_embedding))
        
        self.dep_embeds = nn.Embedding(dep_embedding.shape[0], dep_embedding.shape[0])
        self.dep_embeds.weight.data.copy_(torch.from_numpy(dep_embedding))
        
        self.lstm = nn.LSTM(embedding_dim, hidden_dim // 2,
                            num_layers=2, bidirectional=True,dropout=0.5)

        # Maps the output of the LSTM into tag space.
        self.hidden2tag = nn.Linear(hidden_dim*2, self.tagset_size)

        # Matrix of transition parameters.  Entry i,j is the score of
        # transitioning *to* i *from* j.
        self.transitions = nn.Parameter(
            torch.randn(self.tagset_size, self.tagset_size))

        # These two statements enforce the constraint that we never transfer
        # to the start tag and we never transfer from the stop tag
        self.transitions.data[tag_to_ix[START_TAG], :] = -10000
        self.transitions.data[:, tag_to_ix[STOP_TAG]] = -10000

        self.hidden = self.hidden_initial()
        self.dropout_lstm=nn.Dropout(p=0.5)
        
    def hidden_initial(self):
        return (torch.randn(4, 1, self.hidden_dim // 2).to(device), ### different layers 需要改
                torch.randn(4, 1, self.hidden_dim // 2).to(device)) ###

    def _forward_alg(self, feats):
        # Do the forward algorithm to compute the partition function
        init_alphas = torch.full((1, self.tagset_size), -10000.).to(device)
        # START_TAG has all of the score.
        init_alphas[0][self.tag_to_ix[START_TAG]] = 0.
        #print('_forward_alg')
        # Wrap in a variable so that we will get automatic backprop
        forward_var = init_alphas

        # Iterate through the sentence
        for feat in feats:
            alphas_t = []  # The forward tensors at this timestep
            for next_tag in range(self.tagset_size):
                # broadcast the emission score: it is the same regardless of
                # the previous tag
                emit_score = feat[next_tag].view(
                    1, -1).expand(1, self.tagset_size)
                # the ith entry of trans_score is the score of transitioning to
                # next_tag from i
                trans_score = self.transitions[next_tag].view(1, -1)
                # The ith entry of next_tag_var is the value for the
                # edge (i -> next_tag) before we do log-sum-exp
                next_tag_var = forward_var + trans_score + emit_score
                # The forward variable for this tag is log-sum-exp of all the
                # scores.
                alphas_t.append(log_sum_exp(next_tag_var).view(1))
            forward_var = torch.cat(alphas_t).view(1, -1)
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        alpha = log_sum_exp(terminal_var)
        return alpha
    # method = dot
    def attention(self,lstm_out,hidden_out, hidden_dim, method):
        #print('attention')
        #h_out_T = torch.transpose(hidden_out,0,1)
        h_out_T = torch.transpose(hidden_out,1,2)
        
        h_out_T_1 = h_out_T   
        h_out_0 = hidden_out
        for i in range(lstm_out.size()[0]-1):

          h_out_T = torch.cat((h_out_T, h_out_T_1), 0)
          hidden_out = torch.cat((hidden_out, h_out_0), 0)
        W = nn.Linear(hidden_dim,hidden_dim,bias=False)

        #
        #print('lstm_out',lstm_out.size())
        #print('h_out_T',h_out_T.size())
        #print('hidden_out',hidden_out.size())

        if method == 'dot':
            attention_w = F.softmax(torch.bmm(lstm_out, h_out_T),dim=-1)
        elif method == 'scaled-dot':
            attention_w = F.softmax(torch.bmm(lstm_out, h_out_T)/np.sqrt(self.hidden_dim),dim=-1)
        elif method == 'cosine':
            attention_w = F.softmax(torch.bmm(lstm_out,h_out_T)/(torch.norm(lstm_out)*torch.norm(h_out_T)),dim=-1)

        #elif method == 'general':
        #    attention_w = F.softmax(torch.bmm(lstm_out, W(h_out_T), h_out_T),dim=-1)

        attention_output = torch.bmm(attention_w, hidden_out)
        concat_output = torch.cat((attention_output, lstm_out), 1)  
        return concat_output

    def lstm_ft_get(self, sentence, pos, dep, method):
        self.hidden = self.hidden_initial()
        #print('lstm_ft_get')

        embeds = torch.cat((self.word_embeds(sentence).view(len(sentence), 1, -1),self.pos_embeds(pos).view(len(pos), 1, -1),self.dep_embeds(dep).view(len(dep), 1, -1)),2)
        lstm_out, self.hidden = self.lstm(embeds, self.hidden)


        #print('lstm_out.shape',lstm_out.shape)
        hidden_out = torch.cat((self.hidden[0].view(2, 2, 1, -1)[0,0,:,:],self.hidden[0].view(2, 2, 1, -1)[0,1,:,:]),1) ###################不同layer这里需要改
        hidden_out = hidden_out.unsqueeze(0)
        #print('hidden_out.shape',hidden_out.shape)
        attention_out = self.attention(lstm_out,hidden_out,hidden_dim,method)
        attention_out = attention_out.view(-1,self.hidden_dim*2)

        #print('attention_out.shape',attention_out.shape)
        attention_out = self.dropout_lstm(attention_out)
        lstm_fts = self.hidden2tag(attention_out)
        #print('lstm_fts.shape',lstm_fts.shape)
        return lstm_fts

    def sentence_scoring(self, feats, tags):
        # Gives the score of a provided tag sequence
        score = torch.zeros(1).to(device)
        tags = torch.cat([torch.tensor([self.tag_to_ix[START_TAG]], dtype=torch.long).to(device), tags])
        for i, feat in enumerate(feats):
            score = score + \
                self.transitions[tags[i + 1], tags[i]] + feat[tags[i + 1]]
        score = score + self.transitions[self.tag_to_ix[STOP_TAG], tags[-1]]
        return score

    def viterbi_decoding(self, feats):
        backpointers = []

        # Initialize the viterbi variables in log space
        init_vvars = torch.full((1, self.tagset_size), -10000.).to(device)
        init_vvars[0][self.tag_to_ix[START_TAG]] = 0

        # forward_var at step i holds the viterbi variables for step i-1
        forward_var = init_vvars
        for feat in feats:
            bptrs_t = []  # holds the backpointers for this step
            viterbivars_t = []  # holds the viterbi variables for this step

            for next_tag in range(self.tagset_size):
                # next_tag_var[i] holds the viterbi variable for tag i at the
                # previous step, plus the score of transitioning
                # from tag i to next_tag.
                # We don't include the emission scores here because the max
                # does not depend on them (we add them in below)
                next_tag_var = forward_var + self.transitions[next_tag]
                best_tag_id = argmax(next_tag_var)
                bptrs_t.append(best_tag_id)
                viterbivars_t.append(next_tag_var[0][best_tag_id].view(1))
            # Now add in the emission scores, and assign forward_var to the set
            # of viterbi variables we just computed
            forward_var = (torch.cat(viterbivars_t) + feat).view(1, -1)
            backpointers.append(bptrs_t)

        # Transition to STOP_TAG
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        best_tag_id = argmax(terminal_var)
        path_score = terminal_var[0][best_tag_id]

        # Follow the back pointers to decode the best path.
        best_path = [best_tag_id]
        for bptrs_t in reversed(backpointers):
            best_tag_id = bptrs_t[best_tag_id]
            best_path.append(best_tag_id)
        # Pop off the start tag (we dont want to return that to the caller)
        start = best_path.pop()
        assert start == self.tag_to_ix[START_TAG]  # Sanity check
        best_path.reverse()
        return path_score, best_path

    def neg_ll(self, sentence, tags, pos, dep, method):
        feats = self.lstm_ft_get(sentence, pos, dep, method)
        #print('neg_ll')
        forward_score = self._forward_alg(feats)
        gold_score = self.sentence_scoring(feats, tags)
        return forward_score - gold_score

    #def forward(self, sentence, ent, pos, dep, bert):  # dont confuse this with _forward_alg above.
    def forward(self, sentence, pos, dep):
        #print('forward')
        # Get the emission scores from the BiLSTM

        lstm_fts = self.lstm_ft_get(sentence, pos, dep, method)
        # Find the best path, given the features.
        score, tag_seq = self.viterbi_decoding(lstm_fts)
        return score, tag_seq



def accuracy_calculation(model, input_index, output_index, pos_index, dep_index):

  c, total = 0, 0

  pred_list, truth = [], []

  #print('accuracy_calculation')
for i in range(len(input_index)):
    sentence_in = torch.tensor(input_index[i], dtype=torch.long).to(device)
    
    pos = torch.tensor(pos_index[i], dtype=torch.long).to(device)
    dep = torch.tensor(dep_index[i], dtype=torch.long).to(device)



    _, pred = model.forward(sentence_in, pos, dep)
    for j in range(len(output_index[i])):
      total += 1
      
      pred_list.append(pred[j])
      truth.append(output_index[i][j])

      if pred[j] == output_index[i][j]:
        c += 1

  accuracy = c/total

  return truth, pred_list, accuracy


In [ ]:
def accuracy_calculation(model, input_index, output_index, pos_index, dep_index):

  c, total = 0, 0

  pred_list, truth = [], []

  #print('accuracy_calculation')
  for i in range(len(input_index)):
    sentence_in = torch.tensor(input_index[i], dtype=torch.long).to(device)
    pos = torch.tensor(pos_index[i], dtype=torch.long).to(device)
    dep = torch.tensor(dep_index[i], dtype=torch.long).to(device)



    _, pred = model.forward(sentence_in, pos, dep)
    for j in range(len(output_index[i])):
      total += 1
      
      pred_list.append(pred[j])
      truth.append(output_index[i][j])

      if pred[j] == output_index[i][j]:
        c += 1

  accuracy = c/total

  return truth, pred_list, accuracy

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
hidden_dim = 250

model = BiLSTM_CRF(len(full_word_to_ix), tag_to_ix, embedding_dim, hidden_dim, WV_EMBEDDING_DIM, method='scaled-dot').to(device)
optimizer = optim.SGD(model.parameters(), lr=0.1, weight_decay=1e-4)

import datetime
for epoch in range(2):  
    time1 = datetime.datetime.now()
    train_loss = 0
    method='scaled-dot'
    model.train()
    for i in range(len(train_input_index)):
        tags_index = train_output_index[i]
        dep_index = train_dep_index[i]
        pos_index = train_pos_index[i]
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is,
        # turn them into Tensors of word indices.
        sentence_in = torch.tensor(train_input_index[i], dtype=torch.long).to(device)
        pos = torch.tensor(pos_index, dtype=torch.long).to(device)
        dep = torch.tensor(dep_index, dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)
        # Step 3. Run our forward pass.
        loss = model.neg_ll(sentence_in, targets, pos, dep, method='scaled-dot')
        # Step 4. Compute the loss, gradients, and update the parameters by
        # calling optimizer.step()
        loss.backward()
        optimizer.step()

        train_loss+=loss.item()
    model.eval()
     # 
    _, _, train_acc = accuracy_calculation(model,train_input_index,train_output_index,train_pos_index,train_dep_index)
    _, _, val_acc = accuracy_calculation(model,val_input_index,val_output_index,val_pos_index,val_dep_index)

    val_loss = 0
    for i in range(len(val_input_index)):
        tags_index = val_output_index[i]
        pos_index = val_pos_index[i]
        dep_index = val_dep_index[i]
        sentence_in = torch.tensor(val_input_index[i], dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)
        pos = torch.tensor(pos_index, dtype=torch.long).to(device)
        dep = torch.tensor(dep_index, dtype=torch.long).to(device)
        loss = model.neg_ll(sentence_in, targets, pos, dep, method='scaled-dot')
        val_loss+=loss.item()
    time2 = datetime.datetime.now()

    print("Epoch:%d, Training loss: %.2f, train acc: %.4f, val loss: %.2f, val acc: %.4f, time: %.2fs" %(epoch+1, train_loss,train_acc, val_loss, val_acc, (time2-time1).total_seconds()))

y_pred, y_true, _ = accuracy_calculation(model,val_input_index,val_output_index,val_pos_index,val_dep_index)
def decode_output(output_list):
    ix_to_tag = {v:k for k,v in tag_to_ix.items()}
    return [ix_to_tag[output] for output in output_list]
y_true_decode = decode_output(y_true)
y_pred_decode = decode_output(y_pred)
from sklearn.metrics import classification_report
print(classification_report(y_true_decode,y_pred_decode,digits=4))


Epoch:1, Training loss: 38487.36, train acc: 0.9859, val loss: 2432.55, val acc: 0.9845, time: 754.53s
Epoch:2, Training loss: 6977.49, train acc: 0.9945, val loss: 1671.03, val acc: 0.9921, time: 736.60s
              precision    recall  f1-score   support

           C     0.9567    0.9764    0.9665      1608
           D     0.8844    0.9239    0.9037       381
           O     0.9992    0.9920    0.9956     19123
           P     0.9975    0.9997    0.9986      3927
           S     0.9849    0.9976    0.9912      3280
        SEPA     1.0000    1.0000    1.0000      3603
           T     0.9455    0.9700    0.9576      1432

    accuracy                         0.9918     33354
   macro avg     0.9669    0.9799    0.9733     33354
weighted avg     0.9920    0.9918    0.9919     33354



In [ ]:
import sklearn
from sklearn.metrics import f1_score
sklearn.metrics.f1_score(y_true_decode, y_pred_decode,average='micro')

0.9918450560652395

### Model 5

Calculation method is "scaled dot-product" and position is in the both 2 attention layers (averaged)

In [ ]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import math
from numpy.linalg import norm

torch.manual_seed(1)

def argmax(vec):
    # return the argmax as a python int
    _, idx = torch.max(vec, 1)
    return idx.item()


# Compute log sum exp in a numerically stable way for the forward algorithm
def log_sum_exp(vec):
    max_score = vec[0, argmax(vec)]
    max_score_broadcast = max_score.view(1, -1).expand(1, vec.size()[1])
    return max_score + \
        torch.log(torch.sum(torch.exp(vec - max_score_broadcast)))

class BiLSTM_CRF(nn.Module):

    def __init__(self, vocab_size, tag_to_ix, embedding_dim, hidden_dim, wv_embedding_dim, method):
        super(BiLSTM_CRF, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.tag_to_ix = tag_to_ix
        self.tagset_size = len(tag_to_ix)
        self.wv_embedding_dim = wv_embedding_dim
        self.method = method
        self.word_embeds = nn.Embedding(vocab_size, wv_embedding_dim)
        self.word_embeds.weight.data.copy_(torch.from_numpy(embedding_matrix))
        
        self.pos_embeds = nn.Embedding(pos_embedding.shape[0], pos_embedding.shape[0])
        self.pos_embeds.weight.data.copy_(torch.from_numpy(pos_embedding))
        
        self.dep_embeds = nn.Embedding(dep_embedding.shape[0], dep_embedding.shape[0])
        self.dep_embeds.weight.data.copy_(torch.from_numpy(dep_embedding))
        
        self.lstm = nn.LSTM(embedding_dim, hidden_dim // 2,
                            num_layers=2, bidirectional=True,dropout=0.5)

        # Maps the output of the LSTM into tag space.
        self.hidden2tag = nn.Linear(hidden_dim*2, self.tagset_size)

        # Matrix of transition parameters.  Entry i,j is the score of
        # transitioning *to* i *from* j.
        self.transitions = nn.Parameter(
            torch.randn(self.tagset_size, self.tagset_size))

        # These two statements enforce the constraint that we never transfer
        # to the start tag and we never transfer from the stop tag
        self.transitions.data[tag_to_ix[START_TAG], :] = -10000
        self.transitions.data[:, tag_to_ix[STOP_TAG]] = -10000

        self.hidden = self.hidden_initial()
        self.dropout_lstm=nn.Dropout(p=0.5)
        
    def hidden_initial(self):
        return (torch.randn(4, 1, self.hidden_dim // 2).to(device), ### different layers 需要改
                torch.randn(4, 1, self.hidden_dim // 2).to(device)) ###

    def _forward_alg(self, feats):
        # Do the forward algorithm to compute the partition function
        init_alphas = torch.full((1, self.tagset_size), -10000.).to(device)
        # START_TAG has all of the score.
        init_alphas[0][self.tag_to_ix[START_TAG]] = 0.
        #print('_forward_alg')
        # Wrap in a variable so that we will get automatic backprop
        forward_var = init_alphas

        # Iterate through the sentence
        for feat in feats:
            alphas_t = []  # The forward tensors at this timestep
            for next_tag in range(self.tagset_size):
                # broadcast the emission score: it is the same regardless of
                # the previous tag
                emit_score = feat[next_tag].view(
                    1, -1).expand(1, self.tagset_size)
                # the ith entry of trans_score is the score of transitioning to
                # next_tag from i
                trans_score = self.transitions[next_tag].view(1, -1)
                # The ith entry of next_tag_var is the value for the
                # edge (i -> next_tag) before we do log-sum-exp
                next_tag_var = forward_var + trans_score + emit_score
                # The forward variable for this tag is log-sum-exp of all the
                # scores.
                alphas_t.append(log_sum_exp(next_tag_var).view(1))
            forward_var = torch.cat(alphas_t).view(1, -1)
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        alpha = log_sum_exp(terminal_var)
        return alpha
    # method = dot
    def attention(self,lstm_out,hidden_out, hidden_dim, method):
        #print('attention')
        #h_out_T = torch.transpose(hidden_out,0,1)
        h_out_T = torch.transpose(hidden_out,1,2)
        
        h_out_T_1 = h_out_T   
        h_out_0 = hidden_out
        for i in range(lstm_out.size()[0]-1):

          h_out_T = torch.cat((h_out_T, h_out_T_1), 0)
          hidden_out = torch.cat((hidden_out, h_out_0), 0)
        W = nn.Linear(hidden_dim,hidden_dim,bias=False)

        if method == 'dot':
            attention_w = F.softmax(torch.bmm(lstm_out, h_out_T),dim=-1)
        elif method == 'scaled-dot':
            attention_w = F.softmax(torch.bmm(lstm_out, h_out_T)/np.sqrt(self.hidden_dim),dim=-1)
        elif method == 'cosine':
            attention_w = F.softmax(torch.bmm(lstm_out,h_out_T)/(torch.norm(lstm_out)*torch.norm(h_out_T)),dim=-1)


        attention_output = torch.bmm(attention_w, hidden_out)
        concat_output = torch.cat((attention_output, lstm_out), 1)  
        return concat_output

    def lstm_ft_get(self, sentence, pos, dep, method):
        self.hidden = self.hidden_initial()

        embeds = torch.cat((self.word_embeds(sentence).view(len(sentence), 1, -1),self.pos_embeds(pos).view(len(pos), 1, -1),self.dep_embeds(dep).view(len(dep), 1, -1)),2)
        lstm_out, self.hidden = self.lstm(embeds, self.hidden)

        #print('lstm_out.shape',lstm_out.shape)
        hidden_out = torch.cat((self.hidden[0].view(2, 2, 1, -1)[0,0,:,:],self.hidden[0].view(2, 2, 1, -1)[0,1,:,:]),1) ###################不同layer这里需要改
        hidden_out = hidden_out.unsqueeze(0)
        #print('hidden_out.shape',hidden_out.shape)
        attention_out_1 = self.attention(lstm_out,hidden_out,hidden_dim,method)
        attention_out_1 = attention_out_1.view(-1,self.hidden_dim*2)

        #print('lstm_out.shape',lstm_out.shape)
        hidden_out = torch.cat((self.hidden[0].view(2, 2, 1, -1)[1,0,:,:],self.hidden[0].view(2, 2, 1, -1)[1,1,:,:]),1) ###################不同layer这里需要改
        hidden_out = hidden_out.unsqueeze(0)
        #print('hidden_out.shape',hidden_out.shape)
        attention_out_2 = self.attention(lstm_out,hidden_out,hidden_dim,method)
        attention_out_2 = attention_out_2.view(-1,self.hidden_dim*2)

        w = 0.5
        attention_out = w*attention_out_2 + (1-w)*attention_out_1

        #print('attention_out.shape',attention_out.shape)
        attention_out = self.dropout_lstm(attention_out)
        lstm_fts = self.hidden2tag(attention_out)
        #print('lstm_fts.shape',lstm_fts.shape)
        return lstm_fts

    def sentence_scoring(self, feats, tags):
        # Gives the score of a provided tag sequence
        score = torch.zeros(1).to(device)
        tags = torch.cat([torch.tensor([self.tag_to_ix[START_TAG]], dtype=torch.long).to(device), tags])
        for i, feat in enumerate(feats):
            score = score + \
                self.transitions[tags[i + 1], tags[i]] + feat[tags[i + 1]]
        score = score + self.transitions[self.tag_to_ix[STOP_TAG], tags[-1]]
        return score

    def viterbi_decoding(self, feats):
        backpointers = []

        # Initialize the viterbi variables in log space
        init_vvars = torch.full((1, self.tagset_size), -10000.).to(device)
        init_vvars[0][self.tag_to_ix[START_TAG]] = 0

        # forward_var at step i holds the viterbi variables for step i-1
        forward_var = init_vvars
        for feat in feats:
            bptrs_t = []  # holds the backpointers for this step
            viterbivars_t = []  # holds the viterbi variables for this step

            for next_tag in range(self.tagset_size):
                # next_tag_var[i] holds the viterbi variable for tag i at the
                # previous step, plus the score of transitioning
                # from tag i to next_tag.
                # We don't include the emission scores here because the max
                # does not depend on them (we add them in below)
                next_tag_var = forward_var + self.transitions[next_tag]
                best_tag_id = argmax(next_tag_var)
                bptrs_t.append(best_tag_id)
                viterbivars_t.append(next_tag_var[0][best_tag_id].view(1))
            # Now add in the emission scores, and assign forward_var to the set
            # of viterbi variables we just computed
            forward_var = (torch.cat(viterbivars_t) + feat).view(1, -1)
            backpointers.append(bptrs_t)

        # Transition to STOP_TAG
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        best_tag_id = argmax(terminal_var)
        path_score = terminal_var[0][best_tag_id]

        # Follow the back pointers to decode the best path.
        best_path = [best_tag_id]
        for bptrs_t in reversed(backpointers):
            best_tag_id = bptrs_t[best_tag_id]
            best_path.append(best_tag_id)
        # Pop off the start tag (we dont want to return that to the caller)
        start = best_path.pop()
        assert start == self.tag_to_ix[START_TAG]  # Sanity check
        best_path.reverse()
        return path_score, best_path

    def neg_ll(self, sentence, tags, pos, dep, method):
        feats = self.lstm_ft_get(sentence, pos, dep, method)
        #print('neg_ll')
        forward_score = self._forward_alg(feats)
        gold_score = self.sentence_scoring(feats, tags)
        return forward_score - gold_score

    #def forward(self, sentence, ent, pos, dep, bert):  # dont confuse this with _forward_alg above.
    def forward(self, sentence, pos, dep):
        #print('forward')
        # Get the emission scores from the BiLSTM

        lstm_fts = self.lstm_ft_get(sentence, pos, dep, method)
        # Find the best path, given the features.
        score, tag_seq = self.viterbi_decoding(lstm_fts)
        return score, tag_seq



def accuracy_calculation(model, input_index, output_index, pos_index, dep_index):

  c, total = 0, 0

  pred_list, truth = [], []

  #print('accuracy_calculation')
  for i in range(len(input_index)):
    sentence_in = torch.tensor(input_index[i], dtype=torch.long).to(device)
    pos = torch.tensor(pos_index[i], dtype=torch.long).to(device)
    dep = torch.tensor(dep_index[i], dtype=torch.long).to(device)


    _, pred = model.forward(sentence_in, pos, dep)
    for j in range(len(output_index[i])):
      total += 1
      
      pred_list.append(pred[j])
      truth.append(output_index[i][j])

      if pred[j] == output_index[i][j]:
        c += 1

  accuracy = c/total

  return truth, pred_list, accuracy


In [ ]:
def accuracy_calculation(model, input_index, output_index, pos_index, dep_index):

  c, total = 0, 0

  pred_list, truth = [], []

  #print('accuracy_calculation')
  for i in range(len(input_index)):
    sentence_in = torch.tensor(input_index[i], dtype=torch.long).to(device)
    pos = torch.tensor(pos_index[i], dtype=torch.long).to(device)
    dep = torch.tensor(dep_index[i], dtype=torch.long).to(device)

    _, pred = model.forward(sentence_in, pos, dep)
    for j in range(len(output_index[i])):
      total += 1
      
      pred_list.append(pred[j])
      truth.append(output_index[i][j])

      if pred[j] == output_index[i][j]:
        c += 1

  accuracy = c/total

  return truth, pred_list, accuracy

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
hidden_dim = 250

model = BiLSTM_CRF(len(full_word_to_ix), tag_to_ix, embedding_dim, hidden_dim, WV_EMBEDDING_DIM, method='scaled-dot').to(device)
optimizer = optim.SGD(model.parameters(), lr=0.1, weight_decay=1e-4)

import datetime
for epoch in range(2):  
    time1 = datetime.datetime.now()
    train_loss = 0
    method='scaled-dot'
    model.train()
    for i in range(len(train_input_index)):
        tags_index = train_output_index[i]
        dep_index = train_dep_index[i]
        pos_index = train_pos_index[i]
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is,
        # turn them into Tensors of word indices.
        sentence_in = torch.tensor(train_input_index[i], dtype=torch.long).to(device)
        pos = torch.tensor(pos_index, dtype=torch.long).to(device)
        dep = torch.tensor(dep_index, dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)
        # Step 3. Run our forward pass.
        loss = model.neg_ll(sentence_in, targets, pos, dep, method='scaled-dot')
        # Step 4. Compute the loss, gradients, and update the parameters by
        # calling optimizer.step()
        loss.backward()
        optimizer.step()

        train_loss+=loss.item()
    model.eval()
     # 
    _, _, train_acc = accuracy_calculation(model,train_input_index,train_output_index,train_pos_index,train_dep_index)
    _, _, val_acc = accuracy_calculation(model,val_input_index,val_output_index,val_pos_index,val_dep_index)

    val_loss = 0
    for i in range(len(val_input_index)):
        tags_index = val_output_index[i]
        pos_index = val_pos_index[i]
        dep_index = val_dep_index[i]
        sentence_in = torch.tensor(val_input_index[i], dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)
        pos = torch.tensor(pos_index, dtype=torch.long).to(device)
        dep = torch.tensor(dep_index, dtype=torch.long).to(device)
        loss = model.neg_ll(sentence_in, targets, pos, dep, method='scaled-dot')
        val_loss+=loss.item()
    time2 = datetime.datetime.now()

    print("Epoch:%d, Training loss: %.2f, train acc: %.4f, val loss: %.2f, val acc: %.4f, time: %.2fs" %(epoch+1, train_loss,train_acc, val_loss, val_acc, (time2-time1).total_seconds()))

y_pred, y_true, _ = accuracy_calculation(model,val_input_index,val_output_index,val_pos_index,val_dep_index)
def decode_output(output_list):
    ix_to_tag = {v:k for k,v in tag_to_ix.items()}
    return [ix_to_tag[output] for output in output_list]
y_true_decode = decode_output(y_true)
y_pred_decode = decode_output(y_pred)
from sklearn.metrics import classification_report
print(classification_report(y_true_decode,y_pred_decode,digits=4))


Epoch:1, Training loss: 30269.71, train acc: 0.9852, val loss: 2316.17, val acc: 0.9832, time: 802.12s
Epoch:2, Training loss: 5596.48, train acc: 0.9937, val loss: 1517.98, val acc: 0.9903, time: 778.51s
              precision    recall  f1-score   support

           C     0.9580    0.9638    0.9609      1631
           D     0.7312    0.9151    0.8128       318
           O     0.9991    0.9916    0.9953     19127
           P     0.9964    0.9990    0.9977      3926
           S     0.9862    0.9967    0.9914      3287
        SEPA     1.0000    1.0000    1.0000      3603
           T     0.9489    0.9535    0.9512      1462

    accuracy                         0.9901     33354
   macro avg     0.9457    0.9742    0.9585     33354
weighted avg     0.9908    0.9901    0.9904     33354



In [ ]:
import sklearn
from sklearn.metrics import f1_score
sklearn.metrics.f1_score(y_true_decode, y_pred_decode,average='micro')

0.9901361156083228

## 4.3. Ablation Study 3: Different stacked layers

### Model 1

Numer of layer =1

In [ ]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import math
from numpy.linalg import norm

torch.manual_seed(1)

def argmax(vec):
    # return the argmax as a python int
    _, idx = torch.max(vec, 1)
    return idx.item()


# Compute log sum exp in a numerically stable way for the forward algorithm
def log_sum_exp(vec):
    max_score = vec[0, argmax(vec)]
    max_score_broadcast = max_score.view(1, -1).expand(1, vec.size()[1])
    return max_score + \
        torch.log(torch.sum(torch.exp(vec - max_score_broadcast)))

class BiLSTM_CRF(nn.Module):

    def __init__(self, vocab_size, tag_to_ix, embedding_dim, hidden_dim, wv_embedding_dim, method):
        super(BiLSTM_CRF, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.tag_to_ix = tag_to_ix
        self.tagset_size = len(tag_to_ix)
        self.wv_embedding_dim = wv_embedding_dim
        self.method = method
        self.word_embeds = nn.Embedding(vocab_size, wv_embedding_dim)
        self.word_embeds.weight.data.copy_(torch.from_numpy(embedding_matrix))
        
        self.pos_embeds = nn.Embedding(pos_embedding.shape[0], pos_embedding.shape[0])
        self.pos_embeds.weight.data.copy_(torch.from_numpy(pos_embedding))
        
        self.dep_embeds = nn.Embedding(dep_embedding.shape[0], dep_embedding.shape[0])
        self.dep_embeds.weight.data.copy_(torch.from_numpy(dep_embedding))
        
        self.lstm = nn.LSTM(embedding_dim, hidden_dim // 2,
                            num_layers=1, bidirectional=True,dropout=0.5)

        # Maps the output of the LSTM into tag space.
        self.hidden2tag = nn.Linear(hidden_dim*2, self.tagset_size)

        # Matrix of transition parameters.  Entry i,j is the score of
        # transitioning *to* i *from* j.
        self.transitions = nn.Parameter(
            torch.randn(self.tagset_size, self.tagset_size))

        # These two statements enforce the constraint that we never transfer
        # to the start tag and we never transfer from the stop tag
        self.transitions.data[tag_to_ix[START_TAG], :] = -10000
        self.transitions.data[:, tag_to_ix[STOP_TAG]] = -10000

        self.hidden = self.hidden_initial()
        self.dropout_lstm=nn.Dropout(p=0.5)
        
    def hidden_initial(self):
        return (torch.randn(2, 1, self.hidden_dim // 2).to(device), ### different layers 需要改
                torch.randn(2, 1, self.hidden_dim // 2).to(device)) ###

    def _forward_alg(self, feats):
        # Do the forward algorithm to compute the partition function
        init_alphas = torch.full((1, self.tagset_size), -10000.).to(device)
        # START_TAG has all of the score.
        init_alphas[0][self.tag_to_ix[START_TAG]] = 0.
        #print('_forward_alg')
        # Wrap in a variable so that we will get automatic backprop
        forward_var = init_alphas

        # Iterate through the sentence
        for feat in feats:
            alphas_t = []  # The forward tensors at this timestep
            for next_tag in range(self.tagset_size):
                # broadcast the emission score: it is the same regardless of
                # the previous tag
                emit_score = feat[next_tag].view(
                    1, -1).expand(1, self.tagset_size)
                # the ith entry of trans_score is the score of transitioning to
                # next_tag from i
                trans_score = self.transitions[next_tag].view(1, -1)
                # The ith entry of next_tag_var is the value for the
                # edge (i -> next_tag) before we do log-sum-exp
                next_tag_var = forward_var + trans_score + emit_score
                # The forward variable for this tag is log-sum-exp of all the
                # scores.
                alphas_t.append(log_sum_exp(next_tag_var).view(1))
            forward_var = torch.cat(alphas_t).view(1, -1)
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        alpha = log_sum_exp(terminal_var)
        return alpha
    # method = dot
    def attention(self,lstm_out,hidden_out, hidden_dim, method):

        h_out_T = torch.transpose(hidden_out,1,2)
        
        h_out_T_1 = h_out_T   
        h_out_0 = hidden_out   
        for i in range(lstm_out.size()[0]-1):
          h_out_T = torch.cat((h_out_T, h_out_T_1), 0)
          hidden_out = torch.cat((hidden_out, h_out_0), 0)
        W = nn.Linear(hidden_dim,hidden_dim,bias=False)
        if method == 'dot':
            attention_w = F.softmax(torch.bmm(lstm_out, h_out_T),dim=-1)
        elif method == 'scaled-dot':
            attention_w = F.softmax(torch.bmm(lstm_out, h_out_T)/np.sqrt(self.hidden_dim),dim=-1)
        elif method == 'cosine':
            attention_w = F.softmax(torch.bmm(lstm_out,h_out_T)/(torch.norm(lstm_out)*torch.norm(h_out_T)),dim=-1)

        elif method == 'general':
            attention_w = F.softmax(torch.bmm(lstm_out, W(h_out_T), h_out_T),dim=-1)

        attention_output = torch.bmm(attention_w, hidden_out)
        concat_output = torch.cat((attention_output, lstm_out), 1)
        return concat_output

    def lstm_ft_get(self, sentence, pos, dep, method):
        self.hidden = self.hidden_initial()
        embeds = torch.cat((self.word_embeds(sentence).view(len(sentence), 1, -1),self.pos_embeds(pos).view(len(pos), 1, -1),self.dep_embeds(dep).view(len(dep), 1, -1)),2)
        lstm_out, self.hidden = self.lstm(embeds, self.hidden)
        #print('lstm_out.shape',lstm_out.shape)
        hidden_out = torch.cat((self.hidden[0].view(1, 2, 1, -1)[0,0,:,:],self.hidden[0].view(1, 2, 1, -1)[0,1,:,:]),1) ###################不同layer这里需要改
        hidden_out = hidden_out.unsqueeze(0)
        #print('hidden_out.shape',hidden_out.shape)
        attention_out = self.attention(lstm_out,hidden_out,hidden_dim,method)
        attention_out = attention_out.view(-1,self.hidden_dim*2)
        #print('attention_out.shape',attention_out.shape)
        attention_out = self.dropout_lstm(attention_out)
        lstm_fts = self.hidden2tag(attention_out)
        #print('lstm_fts.shape',lstm_fts.shape)
        return lstm_fts

    def sentence_scoring(self, feats, tags):
        # Gives the score of a provided tag sequence
        score = torch.zeros(1).to(device)
        tags = torch.cat([torch.tensor([self.tag_to_ix[START_TAG]], dtype=torch.long).to(device), tags])
        for i, feat in enumerate(feats):
            score = score + \
                self.transitions[tags[i + 1], tags[i]] + feat[tags[i + 1]]
        score = score + self.transitions[self.tag_to_ix[STOP_TAG], tags[-1]]
        return score

    def viterbi_decoding(self, feats):
        backpointers = []

        # Initialize the viterbi variables in log space
        init_vvars = torch.full((1, self.tagset_size), -10000.).to(device)
        init_vvars[0][self.tag_to_ix[START_TAG]] = 0

        # forward_var at step i holds the viterbi variables for step i-1
        forward_var = init_vvars
        for feat in feats:
            bptrs_t = []  # holds the backpointers for this step
            viterbivars_t = []  # holds the viterbi variables for this step

            for next_tag in range(self.tagset_size):
                # next_tag_var[i] holds the viterbi variable for tag i at the
                # previous step, plus the score of transitioning
                # from tag i to next_tag.
                # We don't include the emission scores here because the max
                # does not depend on them (we add them in below)
                next_tag_var = forward_var + self.transitions[next_tag]
                best_tag_id = argmax(next_tag_var)
                bptrs_t.append(best_tag_id)
                viterbivars_t.append(next_tag_var[0][best_tag_id].view(1))
            # Now add in the emission scores, and assign forward_var to the set
            # of viterbi variables we just computed
            forward_var = (torch.cat(viterbivars_t) + feat).view(1, -1)
            backpointers.append(bptrs_t)

        # Transition to STOP_TAG
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        best_tag_id = argmax(terminal_var)
        path_score = terminal_var[0][best_tag_id]

        # Follow the back pointers to decode the best path.
        best_path = [best_tag_id]
        for bptrs_t in reversed(backpointers):
            best_tag_id = bptrs_t[best_tag_id]
            best_path.append(best_tag_id)
        # Pop off the start tag (we dont want to return that to the caller)
        start = best_path.pop()
        assert start == self.tag_to_ix[START_TAG]  # Sanity check
        best_path.reverse()
        return path_score, best_path

    def neg_ll(self, sentence, tags, pos, dep, method):
        feats = self.lstm_ft_get(sentence, pos, dep, method)
        #print('neg_ll')
        forward_score = self._forward_alg(feats)
        gold_score = self.sentence_scoring(feats, tags)
        return forward_score - gold_score

    #def forward(self, sentence, ent, pos, dep, bert):  # dont confuse this with _forward_alg above.
    def forward(self, sentence, pos, dep):
        #print('forward')
        # Get the emission scores from the BiLSTM

        lstm_fts = self.lstm_ft_get(sentence, pos, dep, method)
        # Find the best path, given the features.
        score, tag_seq = self.viterbi_decoding(lstm_fts)
        return score, tag_seq

def accuracy_calculation(model, input_index, output_index, pos_index, dep_index):

  c, total = 0, 0

  pred_list, truth = [], []

  #print('accuracy_calculation')
  for i in range(len(input_index)):
    sentence_in = torch.tensor(input_index[i], dtype=torch.long).to(device)
    pos = torch.tensor(pos_index[i], dtype=torch.long).to(device)
    dep = torch.tensor(dep_index[i], dtype=torch.long).to(device)

    _, pred = model.forward(sentence_in, pos, dep)
    for j in range(len(output_index[i])):
      total += 1
      pred_list.append(pred[j])
      truth.append(output_index[i][j])
      if pred[j] == output_index[i][j]:
        c += 1
  accuracy = c/total
  return truth, pred_list, accuracy


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
hidden_dim = 250

model = BiLSTM_CRF(len(full_word_to_ix), tag_to_ix, embedding_dim, hidden_dim, WV_EMBEDDING_DIM, method='scaled-dot').to(device)
optimizer = optim.SGD(model.parameters(), lr=0.1, weight_decay=1e-4)

import datetime
for epoch in range(2):  
    time1 = datetime.datetime.now()
    train_loss = 0
    method='scaled-dot'
    model.train()
    for i in range(len(train_input_index)):
        #print(i)
        tags_index = train_output_index[i]
        dep_index = train_dep_index[i]
        pos_index = train_pos_index[i]
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is,
        # turn them into Tensors of word indices.
        sentence_in = torch.tensor(train_input_index[i], dtype=torch.long).to(device)
        pos = torch.tensor(pos_index, dtype=torch.long).to(device)
        dep = torch.tensor(dep_index, dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)
        # Step 3. Run our forward pass.
        loss = model.neg_ll(sentence_in, targets, pos, dep, method='scaled-dot')
        # Step 4. Compute the loss, gradients, and update the parameters by
        # calling optimizer.step()
        loss.backward()
        optimizer.step()

        train_loss+=loss.item()

    model.eval()
     # 
    _, _, train_acc = accuracy_calculation(model,train_input_index,train_output_index,train_pos_index,train_dep_index)
    _, _, val_acc = accuracy_calculation(model,val_input_index,val_output_index,val_pos_index,val_dep_index)

    val_loss = 0
    for i in range(len(val_input_index)):
        tags_index = val_output_index[i]
        pos_index = val_pos_index[i]
        dep_index = val_dep_index[i]
        sentence_in = torch.tensor(val_input_index[i], dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)
        pos = torch.tensor(pos_index, dtype=torch.long).to(device)
        dep = torch.tensor(dep_index, dtype=torch.long).to(device)
        loss = model.neg_ll(sentence_in, targets, pos, dep, method='scaled-dot')
        val_loss+=loss.item()
    time2 = datetime.datetime.now()

    print("Epoch:%d, Training loss: %.2f, train acc: %.4f, val loss: %.2f, val acc: %.4f, time: %.2fs" %(epoch+1, train_loss,train_acc, val_loss, val_acc, (time2-time1).total_seconds()))

y_pred, y_true, _ = accuracy_calculation(model,val_input_index,val_output_index,val_pos_index,val_dep_index)
def decode_output(output_list):
    ix_to_tag = {v:k for k,v in tag_to_ix.items()}
    return [ix_to_tag[output] for output in output_list]
y_true_decode = decode_output(y_true)
y_pred_decode = decode_output(y_pred)
from sklearn.metrics import classification_report
print(classification_report(y_true_decode,y_pred_decode,digits=4))


Epoch:1, Training loss: 24818.84, train acc: 0.9912, val loss: 1663.70, val acc: 0.9886, time: 611.66s
Epoch:2, Training loss: 3723.22, train acc: 0.9979, val loss: 1371.56, val acc: 0.9946, time: 619.91s
              precision    recall  f1-score   support

           C     0.9707    0.9956    0.9830      1600
           D     0.9296    0.9763    0.9524       379
           O     0.9997    0.9926    0.9961     19120
           P     0.9982    0.9995    0.9989      3931
           S     0.9898    0.9994    0.9946      3290
        SEPA     1.0000    1.0000    1.0000      3603
           T     0.9598    0.9853    0.9724      1431

    accuracy                         0.9945     33354
   macro avg     0.9783    0.9927    0.9853     33354
weighted avg     0.9947    0.9945    0.9946     33354



In [ ]:
import sklearn
from sklearn.metrics import f1_score
sklearn.metrics.f1_score(y_true_decode, y_pred_decode,average='micro')

0.9945433831024765

### Model 2

Numer of layers = 2

In [ ]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import math
from numpy.linalg import norm

torch.manual_seed(1)

def argmax(vec):
    # return the argmax as a python int
    _, idx = torch.max(vec, 1)
    return idx.item()


# Compute log sum exp in a numerically stable way for the forward algorithm
def log_sum_exp(vec):
    max_score = vec[0, argmax(vec)]
    max_score_broadcast = max_score.view(1, -1).expand(1, vec.size()[1])
    return max_score + \
        torch.log(torch.sum(torch.exp(vec - max_score_broadcast)))

class BiLSTM_CRF(nn.Module):

    def __init__(self, vocab_size, tag_to_ix, embedding_dim, hidden_dim, wv_embedding_dim, method):
        super(BiLSTM_CRF, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.tag_to_ix = tag_to_ix
        self.tagset_size = len(tag_to_ix)
        self.wv_embedding_dim = wv_embedding_dim
        self.method = method
        self.word_embeds = nn.Embedding(vocab_size, wv_embedding_dim)
        self.word_embeds.weight.data.copy_(torch.from_numpy(embedding_matrix))
        
        self.pos_embeds = nn.Embedding(pos_embedding.shape[0], pos_embedding.shape[0])
        self.pos_embeds.weight.data.copy_(torch.from_numpy(pos_embedding))
        
        self.dep_embeds = nn.Embedding(dep_embedding.shape[0], dep_embedding.shape[0])
        self.dep_embeds.weight.data.copy_(torch.from_numpy(dep_embedding))
        
        self.lstm = nn.LSTM(embedding_dim, hidden_dim // 2,
                            num_layers=2, bidirectional=True,dropout=0.5)

        # Maps the output of the LSTM into tag space.
        self.hidden2tag = nn.Linear(hidden_dim*2, self.tagset_size)

        # Matrix of transition parameters.  Entry i,j is the score of
        # transitioning *to* i *from* j.
        self.transitions = nn.Parameter(
            torch.randn(self.tagset_size, self.tagset_size))

        # These two statements enforce the constraint that we never transfer
        # to the start tag and we never transfer from the stop tag
        self.transitions.data[tag_to_ix[START_TAG], :] = -10000
        self.transitions.data[:, tag_to_ix[STOP_TAG]] = -10000

        self.hidden = self.hidden_initial()
        self.dropout_lstm=nn.Dropout(p=0.5)
        
    def hidden_initial(self):
        return (torch.randn(4, 1, self.hidden_dim // 2).to(device), ### different layers 需要改
                torch.randn(4, 1, self.hidden_dim // 2).to(device)) ###

    def _forward_alg(self, feats):
        # Do the forward algorithm to compute the partition function
        init_alphas = torch.full((1, self.tagset_size), -10000.).to(device)
        # START_TAG has all of the score.
        init_alphas[0][self.tag_to_ix[START_TAG]] = 0.
        #print('_forward_alg')
        # Wrap in a variable so that we will get automatic backprop
        forward_var = init_alphas

        # Iterate through the sentence
        for feat in feats:
            alphas_t = []  # The forward tensors at this timestep
            for next_tag in range(self.tagset_size):
                # broadcast the emission score: it is the same regardless of
                # the previous tag
                emit_score = feat[next_tag].view(
                    1, -1).expand(1, self.tagset_size)
                # the ith entry of trans_score is the score of transitioning to
                # next_tag from i
                trans_score = self.transitions[next_tag].view(1, -1)
                # The ith entry of next_tag_var is the value for the
                # edge (i -> next_tag) before we do log-sum-exp
                next_tag_var = forward_var + trans_score + emit_score
                # The forward variable for this tag is log-sum-exp of all the
                # scores.
                alphas_t.append(log_sum_exp(next_tag_var).view(1))
            forward_var = torch.cat(alphas_t).view(1, -1)
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        alpha = log_sum_exp(terminal_var)
        return alpha
    # method = dot
    def attention(self,lstm_out,hidden_out, hidden_dim, method):
        #print('attention')
        #h_out_T = torch.transpose(hidden_out,0,1)
        h_out_T = torch.transpose(hidden_out,1,2)
        
        h_out_T_1 = h_out_T   
        h_out_0 = hidden_out   
        for i in range(lstm_out.size()[0]-1):
          h_out_T = torch.cat((h_out_T, h_out_T_1), 0)
          hidden_out = torch.cat((hidden_out, h_out_0), 0)
        W = nn.Linear(hidden_dim,hidden_dim,bias=False)
        if method == 'dot':
            attention_w = F.softmax(torch.bmm(lstm_out, h_out_T),dim=-1)
        elif method == 'scaled-dot':
            attention_w = F.softmax(torch.bmm(lstm_out, h_out_T)/np.sqrt(self.hidden_dim),dim=-1)
        elif method == 'cosine':
            attention_w = F.softmax(torch.bmm(lstm_out,h_out_T)/(torch.norm(lstm_out)*torch.norm(h_out_T)),dim=-1)

        elif method == 'general':
            attention_w = F.softmax(torch.bmm(lstm_out, W(h_out_T), h_out_T),dim=-1)

        attention_output = torch.bmm(attention_w, hidden_out)
        concat_output = torch.cat((attention_output, lstm_out), 1)
        return concat_output

    def lstm_ft_get(self, sentence, pos, dep, method):
        self.hidden = self.hidden_initial()
        embeds = torch.cat((self.word_embeds(sentence).view(len(sentence), 1, -1),self.pos_embeds(pos).view(len(pos), 1, -1),self.dep_embeds(dep).view(len(dep), 1, -1)),2)
        lstm_out, self.hidden = self.lstm(embeds, self.hidden)
        #print('lstm_out.shape',lstm_out.shape)
        hidden_out = torch.cat((self.hidden[0].view(2, 2, 1, -1)[1,0,:,:],self.hidden[0].view(2, 2, 1, -1)[1,1,:,:]),1) ###################不同layer这里需要改
        hidden_out = hidden_out.unsqueeze(0)
        #print('hidden_out.shape',hidden_out.shape)
        attention_out = self.attention(lstm_out,hidden_out,hidden_dim,method)
        attention_out = attention_out.view(-1,self.hidden_dim*2)
        #print('attention_out.shape',attention_out.shape)
        attention_out = self.dropout_lstm(attention_out)
        lstm_fts = self.hidden2tag(attention_out)
        #print('lstm_fts.shape',lstm_fts.shape)
        return lstm_fts

    def sentence_scoring(self, feats, tags):
        # Gives the score of a provided tag sequence
        score = torch.zeros(1).to(device)
        tags = torch.cat([torch.tensor([self.tag_to_ix[START_TAG]], dtype=torch.long).to(device), tags])
        for i, feat in enumerate(feats):
            score = score + \
                self.transitions[tags[i + 1], tags[i]] + feat[tags[i + 1]]
        score = score + self.transitions[self.tag_to_ix[STOP_TAG], tags[-1]]
        return score

    def viterbi_decoding(self, feats):
        backpointers = []

        # Initialize the viterbi variables in log space
        init_vvars = torch.full((1, self.tagset_size), -10000.).to(device)
        init_vvars[0][self.tag_to_ix[START_TAG]] = 0

        # forward_var at step i holds the viterbi variables for step i-1
        forward_var = init_vvars
        for feat in feats:
            bptrs_t = []  # holds the backpointers for this step
            viterbivars_t = []  # holds the viterbi variables for this step

            for next_tag in range(self.tagset_size):
                # next_tag_var[i] holds the viterbi variable for tag i at the
                # previous step, plus the score of transitioning
                # from tag i to next_tag.
                # We don't include the emission scores here because the max
                # does not depend on them (we add them in below)
                next_tag_var = forward_var + self.transitions[next_tag]
                best_tag_id = argmax(next_tag_var)
                bptrs_t.append(best_tag_id)
                viterbivars_t.append(next_tag_var[0][best_tag_id].view(1))
            # Now add in the emission scores, and assign forward_var to the set
            # of viterbi variables we just computed
            forward_var = (torch.cat(viterbivars_t) + feat).view(1, -1)
            backpointers.append(bptrs_t)

        # Transition to STOP_TAG
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        best_tag_id = argmax(terminal_var)
        path_score = terminal_var[0][best_tag_id]

        # Follow the back pointers to decode the best path.
        best_path = [best_tag_id]
        for bptrs_t in reversed(backpointers):
            best_tag_id = bptrs_t[best_tag_id]
            best_path.append(best_tag_id)
        # Pop off the start tag (we dont want to return that to the caller)
        start = best_path.pop()
        assert start == self.tag_to_ix[START_TAG]  # Sanity check
        best_path.reverse()
        return path_score, best_path

    def neg_ll(self, sentence, tags, pos, dep, method):
        feats = self.lstm_ft_get(sentence, pos, dep, method)
        #print('neg_ll')
        forward_score = self._forward_alg(feats)
        gold_score = self.sentence_scoring(feats, tags)
        return forward_score - gold_score

    #def forward(self, sentence, ent, pos, dep, bert):  # dont confuse this with _forward_alg above.
    def forward(self, sentence, pos, dep):
        #print('forward')
        # Get the emission scores from the BiLSTM

        lstm_fts = self.lstm_ft_get(sentence, pos, dep, method)
        # Find the best path, given the features.
        score, tag_seq = self.viterbi_decoding(lstm_fts)
        return score, tag_seq



def accuracy_calculation(model, input_index, output_index, pos_index, dep_index):

  c, total = 0, 0

  pred_list, truth = [], []

  #print('accuracy_calculation')
  for i in range(len(input_index)):
    sentence_in = torch.tensor(input_index[i], dtype=torch.long).to(device)
    
    pos = torch.tensor(pos_index[i], dtype=torch.long).to(device)
    dep = torch.tensor(dep_index[i], dtype=torch.long).to(device)



    _, pred = model.forward(sentence_in, pos, dep)
    for j in range(len(output_index[i])):
      total += 1
      
      pred_list.append(pred[j])
      truth.append(output_index[i][j])

      if pred[j] == output_index[i][j]:
        c += 1

  accuracy = c/total

  return truth, pred_list, accuracy


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
hidden_dim = 250

model = BiLSTM_CRF(len(full_word_to_ix), tag_to_ix, embedding_dim, hidden_dim, WV_EMBEDDING_DIM, method='scaled-dot').to(device)
optimizer = optim.SGD(model.parameters(), lr=0.1, weight_decay=1e-4)

import datetime
for epoch in range(2):  
    time1 = datetime.datetime.now()
    train_loss = 0
    method='scaled-dot'
    model.train()
    for i in range(len(train_input_index)):
        #print(i)
        tags_index = train_output_index[i]
        dep_index = train_dep_index[i]
        pos_index = train_pos_index[i]
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is,
        # turn them into Tensors of word indices.
        sentence_in = torch.tensor(train_input_index[i], dtype=torch.long).to(device)
        pos = torch.tensor(pos_index, dtype=torch.long).to(device)
        dep = torch.tensor(dep_index, dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)
        # Step 3. Run our forward pass.
        loss = model.neg_ll(sentence_in, targets, pos, dep, method='scaled-dot')
        # Step 4. Compute the loss, gradients, and update the parameters by
        # calling optimizer.step()
        loss.backward()
        optimizer.step()

        train_loss+=loss.item()

    model.eval()
     # 
    _, _, train_acc = accuracy_calculation(model,train_input_index,train_output_index,train_pos_index,train_dep_index)
    _, _, val_acc = accuracy_calculation(model,val_input_index,val_output_index,val_pos_index,val_dep_index)

    val_loss = 0
    for i in range(len(val_input_index)):
        tags_index = val_output_index[i]
        pos_index = val_pos_index[i]
        dep_index = val_dep_index[i]
        sentence_in = torch.tensor(val_input_index[i], dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)
        pos = torch.tensor(pos_index, dtype=torch.long).to(device)
        dep = torch.tensor(dep_index, dtype=torch.long).to(device)
        loss = model.neg_ll(sentence_in, targets, pos, dep, method='scaled-dot')
        val_loss+=loss.item()
    time2 = datetime.datetime.now()

    print("Epoch:%d, Training loss: %.2f, train acc: %.4f, val loss: %.2f, val acc: %.4f, time: %.2fs" %(epoch+1, train_loss,train_acc, val_loss, val_acc, (time2-time1).total_seconds()))

y_pred, y_true, _ = accuracy_calculation(model,val_input_index,val_output_index,val_pos_index,val_dep_index)
def decode_output(output_list):
    ix_to_tag = {v:k for k,v in tag_to_ix.items()}
    return [ix_to_tag[output] for output in output_list]
y_true_decode = decode_output(y_true)
y_pred_decode = decode_output(y_pred)
from sklearn.metrics import classification_report
print(classification_report(y_true_decode,y_pred_decode,digits=4))

Epoch:1, Training loss: 34709.38, train acc: 0.9865, val loss: 2175.60, val acc: 0.9844, time: 630.77s
Epoch:2, Training loss: 5511.21, train acc: 0.9958, val loss: 1519.26, val acc: 0.9933, time: 619.77s
              precision    recall  f1-score   support

           C     0.9677    0.9802    0.9739      1620
           D     0.9045    0.9863    0.9436       365
           O     0.9997    0.9921    0.9959     19130
           P     0.9970    0.9997    0.9983      3925
           S     0.9831    0.9973    0.9901      3275
        SEPA     1.0000    1.0000    1.0000      3603
           T     0.9598    0.9819    0.9707      1436

    accuracy                         0.9933     33354
   macro avg     0.9731    0.9911    0.9818     33354
weighted avg     0.9935    0.9933    0.9933     33354



In [ ]:
import sklearn
from sklearn.metrics import f1_score
sklearn.metrics.f1_score(y_true_decode, y_pred_decode,average='micro')

0.9932841638184325

### Model 3

Number of layers = 3

In [ ]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import math
from numpy.linalg import norm

torch.manual_seed(1)

def argmax(vec):
    # return the argmax as a python int
    _, idx = torch.max(vec, 1)
    return idx.item()


# Compute log sum exp in a numerically stable way for the forward algorithm
def log_sum_exp(vec):
    max_score = vec[0, argmax(vec)]
    max_score_broadcast = max_score.view(1, -1).expand(1, vec.size()[1])
    return max_score + \
        torch.log(torch.sum(torch.exp(vec - max_score_broadcast)))

class BiLSTM_CRF(nn.Module):

    def __init__(self, vocab_size, tag_to_ix, embedding_dim, hidden_dim, wv_embedding_dim, method):
        super(BiLSTM_CRF, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.tag_to_ix = tag_to_ix
        self.tagset_size = len(tag_to_ix)
        self.wv_embedding_dim = wv_embedding_dim
        self.method = method
        self.word_embeds = nn.Embedding(vocab_size, wv_embedding_dim)
        self.word_embeds.weight.data.copy_(torch.from_numpy(embedding_matrix))
        
        self.pos_embeds = nn.Embedding(pos_embedding.shape[0], pos_embedding.shape[0])
        self.pos_embeds.weight.data.copy_(torch.from_numpy(pos_embedding))
        
        self.dep_embeds = nn.Embedding(dep_embedding.shape[0], dep_embedding.shape[0])
        self.dep_embeds.weight.data.copy_(torch.from_numpy(dep_embedding))
        
        self.lstm = nn.LSTM(embedding_dim, hidden_dim // 2,
                            num_layers=3, bidirectional=True,dropout=0.5)

        # Maps the output of the LSTM into tag space.
        self.hidden2tag = nn.Linear(hidden_dim*2, self.tagset_size)

        # Matrix of transition parameters.  Entry i,j is the score of
        # transitioning *to* i *from* j.
        self.transitions = nn.Parameter(
            torch.randn(self.tagset_size, self.tagset_size))

        # These two statements enforce the constraint that we never transfer
        # to the start tag and we never transfer from the stop tag
        self.transitions.data[tag_to_ix[START_TAG], :] = -10000
        self.transitions.data[:, tag_to_ix[STOP_TAG]] = -10000

        self.hidden = self.hidden_initial()
        self.dropout_lstm=nn.Dropout(p=0.5)
        
    def hidden_initial(self):
        return (torch.randn(6, 1, self.hidden_dim // 2).to(device), ### different layers 需要改
                torch.randn(6, 1, self.hidden_dim // 2).to(device)) ###

    def _forward_alg(self, feats):
        # Do the forward algorithm to compute the partition function
        init_alphas = torch.full((1, self.tagset_size), -10000.).to(device)
        # START_TAG has all of the score.
        init_alphas[0][self.tag_to_ix[START_TAG]] = 0.
        #print('_forward_alg')
        # Wrap in a variable so that we will get automatic backprop
        forward_var = init_alphas

        # Iterate through the sentence
        for feat in feats:
            alphas_t = []  # The forward tensors at this timestep
            for next_tag in range(self.tagset_size):
                # broadcast the emission score: it is the same regardless of
                # the previous tag
                emit_score = feat[next_tag].view(
                    1, -1).expand(1, self.tagset_size)
                # the ith entry of trans_score is the score of transitioning to
                # next_tag from i
                trans_score = self.transitions[next_tag].view(1, -1)
                # The ith entry of next_tag_var is the value for the
                # edge (i -> next_tag) before we do log-sum-exp
                next_tag_var = forward_var + trans_score + emit_score
                # The forward variable for this tag is log-sum-exp of all the
                # scores.
                alphas_t.append(log_sum_exp(next_tag_var).view(1))
            forward_var = torch.cat(alphas_t).view(1, -1)
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        alpha = log_sum_exp(terminal_var)
        return alpha
    # method = dot
    def attention(self,lstm_out,hidden_out, hidden_dim, method):

        h_out_T = torch.transpose(hidden_out,1,2)
        
        h_out_T_1 = h_out_T   
        h_out_0 = hidden_out   
        for i in range(lstm_out.size()[0]-1):
          h_out_T = torch.cat((h_out_T, h_out_T_1), 0)
          hidden_out = torch.cat((hidden_out, h_out_0), 0)
        W = nn.Linear(hidden_dim,hidden_dim,bias=False)
        if method == 'dot':
            attention_w = F.softmax(torch.bmm(lstm_out, h_out_T),dim=-1)
        elif method == 'scaled-dot':
            attention_w = F.softmax(torch.bmm(lstm_out, h_out_T)/np.sqrt(self.hidden_dim),dim=-1)
        elif method == 'cosine':
            attention_w = F.softmax(torch.bmm(lstm_out,h_out_T)/(torch.norm(lstm_out)*torch.norm(h_out_T)),dim=-1)

        elif method == 'general':
            attention_w = F.softmax(torch.bmm(lstm_out, W(h_out_T), h_out_T),dim=-1)

        attention_output = torch.bmm(attention_w, hidden_out)
        concat_output = torch.cat((attention_output, lstm_out), 1)
        return concat_output

    def lstm_ft_get(self, sentence, pos, dep, method):
        self.hidden = self.hidden_initial()
        embeds = torch.cat((self.word_embeds(sentence).view(len(sentence), 1, -1),self.pos_embeds(pos).view(len(pos), 1, -1),self.dep_embeds(dep).view(len(dep), 1, -1)),2)
        lstm_out, self.hidden = self.lstm(embeds, self.hidden)
        #print('lstm_out.shape',lstm_out.shape)
        hidden_out = torch.cat((self.hidden[0].view(3, 2, 1, -1)[1,0,:,:],self.hidden[0].view(3, 2, 1, -1)[1,1,:,:]),1) ###################不同layer这里需要改
        hidden_out = hidden_out.unsqueeze(0)
        #print('hidden_out.shape',hidden_out.shape)
        attention_out = self.attention(lstm_out,hidden_out,hidden_dim,method)
        attention_out = attention_out.view(-1,self.hidden_dim*2)
        #print('attention_out.shape',attention_out.shape)
        attention_out = self.dropout_lstm(attention_out)
        lstm_fts = self.hidden2tag(attention_out)
        #print('lstm_fts.shape',lstm_fts.shape)
        return lstm_fts

    def sentence_scoring(self, feats, tags):
        # Gives the score of a provided tag sequence
        score = torch.zeros(1).to(device)
        tags = torch.cat([torch.tensor([self.tag_to_ix[START_TAG]], dtype=torch.long).to(device), tags])
        for i, feat in enumerate(feats):
            score = score + \
                self.transitions[tags[i + 1], tags[i]] + feat[tags[i + 1]]
        score = score + self.transitions[self.tag_to_ix[STOP_TAG], tags[-1]]
        return score

    def viterbi_decoding(self, feats):
        backpointers = []

        # Initialize the viterbi variables in log space
        init_vvars = torch.full((1, self.tagset_size), -10000.).to(device)
        init_vvars[0][self.tag_to_ix[START_TAG]] = 0

        # forward_var at step i holds the viterbi variables for step i-1
        forward_var = init_vvars
        for feat in feats:
            bptrs_t = []  # holds the backpointers for this step
            viterbivars_t = []  # holds the viterbi variables for this step

            for next_tag in range(self.tagset_size):
                # next_tag_var[i] holds the viterbi variable for tag i at the
                # previous step, plus the score of transitioning
                # from tag i to next_tag.
                # We don't include the emission scores here because the max
                # does not depend on them (we add them in below)
                next_tag_var = forward_var + self.transitions[next_tag]
                best_tag_id = argmax(next_tag_var)
                bptrs_t.append(best_tag_id)
                viterbivars_t.append(next_tag_var[0][best_tag_id].view(1))
            # Now add in the emission scores, and assign forward_var to the set
            # of viterbi variables we just computed
            forward_var = (torch.cat(viterbivars_t) + feat).view(1, -1)
            backpointers.append(bptrs_t)

        # Transition to STOP_TAG
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        best_tag_id = argmax(terminal_var)
        path_score = terminal_var[0][best_tag_id]

        # Follow the back pointers to decode the best path.
        best_path = [best_tag_id]
        for bptrs_t in reversed(backpointers):
            best_tag_id = bptrs_t[best_tag_id]
            best_path.append(best_tag_id)
        # Pop off the start tag (we dont want to return that to the caller)
        start = best_path.pop()
        assert start == self.tag_to_ix[START_TAG]  # Sanity check
        best_path.reverse()
        return path_score, best_path

    def neg_ll(self, sentence, tags, pos, dep, method):
        feats = self.lstm_ft_get(sentence, pos, dep, method)
        #print('neg_ll')
        forward_score = self._forward_alg(feats)
        gold_score = self.sentence_scoring(feats, tags)
        return forward_score - gold_score

    #def forward(self, sentence, ent, pos, dep, bert):  # dont confuse this with _forward_alg above.
    def forward(self, sentence, pos, dep):
        #print('forward')
        # Get the emission scores from the BiLSTM

        lstm_fts = self.lstm_ft_get(sentence, pos, dep, method)
        # Find the best path, given the features.
        score, tag_seq = self.viterbi_decoding(lstm_fts)
        return score, tag_seq



def accuracy_calculation(model, input_index, output_index, pos_index, dep_index):

  c, total = 0, 0

  pred_list, truth = [], []

  #print('accuracy_calculation')
for i in range(len(input_index)):
    sentence_in = torch.tensor(input_index[i], dtype=torch.long).to(device)
    
    pos = torch.tensor(pos_index[i], dtype=torch.long).to(device)
    dep = torch.tensor(dep_index[i], dtype=torch.long).to(device)



    _, pred = model.forward(sentence_in, pos, dep)
    for j in range(len(output_index[i])):
      total += 1
      
      pred_list.append(pred[j])
      truth.append(output_index[i][j])

      if pred[j] == output_index[i][j]:
        c += 1

  accuracy = c/total

  return truth, pred_list, accuracy


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
hidden_dim = 250

model = BiLSTM_CRF(len(full_word_to_ix), tag_to_ix, embedding_dim, hidden_dim, WV_EMBEDDING_DIM, method='scaled-dot').to(device)
optimizer = optim.SGD(model.parameters(), lr=0.1, weight_decay=1e-4)

import datetime
for epoch in range(2):  
    time1 = datetime.datetime.now()
    train_loss = 0
    method='scaled-dot'
    model.train()
    for i in range(len(train_input_index)):
        #print(i)
        tags_index = train_output_index[i]
        dep_index = train_dep_index[i]
        pos_index = train_pos_index[i]
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is,
        # turn them into Tensors of word indices.
        sentence_in = torch.tensor(train_input_index[i], dtype=torch.long).to(device)
        pos = torch.tensor(pos_index, dtype=torch.long).to(device)
        dep = torch.tensor(dep_index, dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)
        # Step 3. Run our forward pass.
        loss = model.neg_ll(sentence_in, targets, pos, dep, method='scaled-dot')
        # Step 4. Compute the loss, gradients, and update the parameters by
        # calling optimizer.step()
        loss.backward()
        optimizer.step()

        train_loss+=loss.item()

    model.eval()
     # 
    _, _, train_acc = accuracy_calculation(model,train_input_index,train_output_index,train_pos_index,train_dep_index)
    _, _, val_acc = accuracy_calculation(model,val_input_index,val_output_index,val_pos_index,val_dep_index)

    val_loss = 0
    for i in range(len(val_input_index)):
        tags_index = val_output_index[i]
        pos_index = val_pos_index[i]
        dep_index = val_dep_index[i]
        sentence_in = torch.tensor(val_input_index[i], dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)
        pos = torch.tensor(pos_index, dtype=torch.long).to(device)
        dep = torch.tensor(dep_index, dtype=torch.long).to(device)
        loss = model.neg_ll(sentence_in, targets, pos, dep, method='scaled-dot')
        val_loss+=loss.item()
    time2 = datetime.datetime.now()

    print("Epoch:%d, Training loss: %.2f, train acc: %.4f, val loss: %.2f, val acc: %.4f, time: %.2fs" %(epoch+1, train_loss,train_acc, val_loss, val_acc, (time2-time1).total_seconds()))

y_pred, y_true, _ = accuracy_calculation(model,val_input_index,val_output_index,val_pos_index,val_dep_index)
def decode_output(output_list):
    ix_to_tag = {v:k for k,v in tag_to_ix.items()}
    return [ix_to_tag[output] for output in output_list]
y_true_decode = decode_output(y_true)
y_pred_decode = decode_output(y_pred)
from sklearn.metrics import classification_report
print(classification_report(y_true_decode,y_pred_decode,digits=4))


Epoch:1, Training loss: 63081.22, train acc: 0.9528, val loss: 6157.26, val acc: 0.9524, time: 664.89s
Epoch:2, Training loss: 16728.59, train acc: 0.9836, val loss: 2717.06, val acc: 0.9822, time: 667.52s
              precision    recall  f1-score   support

           C     0.9366    0.9505    0.9435      1617
           D     0.6734    0.7813    0.7233       343
           O     0.9981    0.9848    0.9914     19240
           P     0.9863    0.9992    0.9927      3885
           S     0.9600    0.9953    0.9773      3204
        SEPA     1.0000    1.0000    1.0000      3603
           T     0.9176    0.9220    0.9198      1462

    accuracy                         0.9826     33354
   macro avg     0.9246    0.9476    0.9354     33354
weighted avg     0.9834    0.9826    0.9829     33354



In [ ]:
import sklearn
from sklearn.metrics import f1_score
sklearn.metrics.f1_score(y_true_decode, y_pred_decode,average='micro')

0.9826407627271092

Number of layers = 3

## 4.4. Ablation Study 4: with/without CRF

### Model 1 With CRF

In [ ]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import math
from numpy.linalg import norm

torch.manual_seed(1)

def argmax(vec):
    # return the argmax as a python int
    _, idx = torch.max(vec, 1)
    return idx.item()


# Compute log sum exp in a numerically stable way for the forward algorithm
def log_sum_exp(vec):
    max_score = vec[0, argmax(vec)]
    max_score_broadcast = max_score.view(1, -1).expand(1, vec.size()[1])
    return max_score + \
        torch.log(torch.sum(torch.exp(vec - max_score_broadcast)))

class BiLSTM_CRF(nn.Module):

    def __init__(self, vocab_size, tag_to_ix, embedding_dim, hidden_dim, wv_embedding_dim, method):
        super(BiLSTM_CRF, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.tag_to_ix = tag_to_ix
        self.tagset_size = len(tag_to_ix)
        self.wv_embedding_dim = wv_embedding_dim
        self.method = method
        self.word_embeds = nn.Embedding(vocab_size, wv_embedding_dim)
        self.word_embeds.weight.data.copy_(torch.from_numpy(embedding_matrix))
        
        self.pos_embeds = nn.Embedding(pos_embedding.shape[0], pos_embedding.shape[0])
        self.pos_embeds.weight.data.copy_(torch.from_numpy(pos_embedding))
        
        self.dep_embeds = nn.Embedding(dep_embedding.shape[0], dep_embedding.shape[0])
        self.dep_embeds.weight.data.copy_(torch.from_numpy(dep_embedding))
        
        self.lstm = nn.LSTM(embedding_dim, hidden_dim // 2,
                            num_layers=1, bidirectional=True,dropout=0.5)

        # Maps the output of the LSTM into tag space.
        self.hidden2tag = nn.Linear(hidden_dim*2, self.tagset_size)

        # Matrix of transition parameters.  Entry i,j is the score of
        # transitioning *to* i *from* j.
        self.transitions = nn.Parameter(
            torch.randn(self.tagset_size, self.tagset_size))

        # These two statements enforce the constraint that we never transfer
        # to the start tag and we never transfer from the stop tag
        self.transitions.data[tag_to_ix[START_TAG], :] = -10000
        self.transitions.data[:, tag_to_ix[STOP_TAG]] = -10000

        self.hidden = self.hidden_initial()
        self.dropout_lstm=nn.Dropout(p=0.5)
        
    def hidden_initial(self):
        return (torch.randn(2, 1, self.hidden_dim // 2).to(device), ### different layers 需要改
                torch.randn(2, 1, self.hidden_dim // 2).to(device)) ###

    def _forward_alg(self, feats):
        # Do the forward algorithm to compute the partition function
        init_alphas = torch.full((1, self.tagset_size), -10000.).to(device)
        # START_TAG has all of the score.
        init_alphas[0][self.tag_to_ix[START_TAG]] = 0.
        #print('_forward_alg')
        # Wrap in a variable so that we will get automatic backprop
        forward_var = init_alphas

        # Iterate through the sentence
        for feat in feats:
            alphas_t = []  # The forward tensors at this timestep
            for next_tag in range(self.tagset_size):
                # broadcast the emission score: it is the same regardless of
                # the previous tag
                emit_score = feat[next_tag].view(
                    1, -1).expand(1, self.tagset_size)
                # the ith entry of trans_score is the score of transitioning to
                # next_tag from i
                trans_score = self.transitions[next_tag].view(1, -1)
                # The ith entry of next_tag_var is the value for the
                # edge (i -> next_tag) before we do log-sum-exp
                next_tag_var = forward_var + trans_score + emit_score
                # The forward variable for this tag is log-sum-exp of all the
                # scores.
                alphas_t.append(log_sum_exp(next_tag_var).view(1))
            forward_var = torch.cat(alphas_t).view(1, -1)
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        alpha = log_sum_exp(terminal_var)
        return alpha
    # method = dot
    def attention(self,lstm_out,hidden_out, hidden_dim, method):

        h_out_T = torch.transpose(hidden_out,1,2)
        
        h_out_T_1 = h_out_T   
        h_out_0 = hidden_out   
        for i in range(lstm_out.size()[0]-1):
          h_out_T = torch.cat((h_out_T, h_out_T_1), 0)
          hidden_out = torch.cat((hidden_out, h_out_0), 0)
        W = nn.Linear(hidden_dim,hidden_dim,bias=False)
        if method == 'dot':
            attention_w = F.softmax(torch.bmm(lstm_out, h_out_T),dim=-1)
        elif method == 'scaled-dot':
            attention_w = F.softmax(torch.bmm(lstm_out, h_out_T)/np.sqrt(self.hidden_dim),dim=-1)
        elif method == 'cosine':
            attention_w = F.softmax(torch.bmm(lstm_out,h_out_T)/(torch.norm(lstm_out)*torch.norm(h_out_T)),dim=-1)

        elif method == 'general':
            attention_w = F.softmax(torch.bmm(lstm_out, W(h_out_T), h_out_T),dim=-1)

        attention_output = torch.bmm(attention_w, hidden_out)
        concat_output = torch.cat((attention_output, lstm_out), 1)
        return concat_output

    def lstm_ft_get(self, sentence, pos, dep, method):
        self.hidden = self.hidden_initial()
        embeds = torch.cat((self.word_embeds(sentence).view(len(sentence), 1, -1),self.pos_embeds(pos).view(len(pos), 1, -1),self.dep_embeds(dep).view(len(dep), 1, -1)),2)
        lstm_out, self.hidden = self.lstm(embeds, self.hidden)
        #print('lstm_out.shape',lstm_out.shape)
        hidden_out = torch.cat((self.hidden[0].view(1, 2, 1, -1)[0,0,:,:],self.hidden[0].view(1, 2, 1, -1)[0,1,:,:]),1) ###################不同layer这里需要改
        hidden_out = hidden_out.unsqueeze(0)
        #print('hidden_out.shape',hidden_out.shape)
        attention_out = self.attention(lstm_out,hidden_out,hidden_dim,method)
        attention_out = attention_out.view(-1,self.hidden_dim*2)
        #print('attention_out.shape',attention_out.shape)
        attention_out = self.dropout_lstm(attention_out)
        lstm_fts = self.hidden2tag(attention_out)
        #print('lstm_fts.shape',lstm_fts.shape)
        return lstm_fts

    def sentence_scoring(self, feats, tags):
        # Gives the score of a provided tag sequence
        score = torch.zeros(1).to(device)
        tags = torch.cat([torch.tensor([self.tag_to_ix[START_TAG]], dtype=torch.long).to(device), tags])
        for i, feat in enumerate(feats):
            score = score + \
                self.transitions[tags[i + 1], tags[i]] + feat[tags[i + 1]]
        score = score + self.transitions[self.tag_to_ix[STOP_TAG], tags[-1]]
        return score

    def viterbi_decoding(self, feats):
        backpointers = []

        # Initialize the viterbi variables in log space
        init_vvars = torch.full((1, self.tagset_size), -10000.).to(device)
        init_vvars[0][self.tag_to_ix[START_TAG]] = 0

        # forward_var at step i holds the viterbi variables for step i-1
        forward_var = init_vvars
        for feat in feats:
            bptrs_t = []  # holds the backpointers for this step
            viterbivars_t = []  # holds the viterbi variables for this step

            for next_tag in range(self.tagset_size):
                # next_tag_var[i] holds the viterbi variable for tag i at the
                # previous step, plus the score of transitioning
                # from tag i to next_tag.
                # We don't include the emission scores here because the max
                # does not depend on them (we add them in below)
                next_tag_var = forward_var + self.transitions[next_tag]
                best_tag_id = argmax(next_tag_var)
                bptrs_t.append(best_tag_id)
                viterbivars_t.append(next_tag_var[0][best_tag_id].view(1))
            # Now add in the emission scores, and assign forward_var to the set
            # of viterbi variables we just computed
            forward_var = (torch.cat(viterbivars_t) + feat).view(1, -1)
            backpointers.append(bptrs_t)

        # Transition to STOP_TAG
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        best_tag_id = argmax(terminal_var)
        path_score = terminal_var[0][best_tag_id]

        # Follow the back pointers to decode the best path.
        best_path = [best_tag_id]
        for bptrs_t in reversed(backpointers):
            best_tag_id = bptrs_t[best_tag_id]
            best_path.append(best_tag_id)
        # Pop off the start tag (we dont want to return that to the caller)
        start = best_path.pop()
        assert start == self.tag_to_ix[START_TAG]  # Sanity check
        best_path.reverse()
        return path_score, best_path

    def neg_ll(self, sentence, tags, pos, dep, method):
        feats = self.lstm_ft_get(sentence, pos, dep, method)
        #print('neg_ll')
        forward_score = self._forward_alg(feats)
        gold_score = self.sentence_scoring(feats, tags)
        return forward_score - gold_score

    #def forward(self, sentence, ent, pos, dep, bert):  # dont confuse this with _forward_alg above.
    def forward(self, sentence, pos, dep):
        #print('forward')
        # Get the emission scores from the BiLSTM

        lstm_fts = self.lstm_ft_get(sentence, pos, dep, method)
        # Find the best path, given the features.
        score, tag_seq = self.viterbi_decoding(lstm_fts)
        return score, tag_seq



def accuracy_calculation(model, input_index, output_index, pos_index, dep_index):

  c, total = 0, 0

  pred_list, truth = [], []

  #print('accuracy_calculation')
  for i in range(len(input_index)):
    sentence_in = torch.tensor(input_index[i], dtype=torch.long).to(device)
    
    pos = torch.tensor(pos_index[i], dtype=torch.long).to(device)
    dep = torch.tensor(dep_index[i], dtype=torch.long).to(device)



    _, pred = model.forward(sentence_in, pos, dep)
    for j in range(len(output_index[i])):
      total += 1
      
      pred_list.append(pred[j])
      truth.append(output_index[i][j])

      if pred[j] == output_index[i][j]:
        c += 1

  accuracy = c/total

  return truth, pred_list, accuracy


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
hidden_dim = 250

model = BiLSTM_CRF(len(full_word_to_ix), tag_to_ix, embedding_dim, hidden_dim, WV_EMBEDDING_DIM, method='scaled-dot').to(device)
optimizer = optim.SGD(model.parameters(), lr=0.1, weight_decay=1e-4)

import datetime
for epoch in range(2):  
    time1 = datetime.datetime.now()
    train_loss = 0
    method='scaled-dot'
    model.train()
    for i in range(len(train_input_index)):
        #print(i)
        tags_index = train_output_index[i]
        dep_index = train_dep_index[i]
        pos_index = train_pos_index[i]
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is,
        # turn them into Tensors of word indices.
        sentence_in = torch.tensor(train_input_index[i], dtype=torch.long).to(device)
        pos = torch.tensor(pos_index, dtype=torch.long).to(device)
        dep = torch.tensor(dep_index, dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)
        # Step 3. Run our forward pass.
        loss = model.neg_ll(sentence_in, targets, pos, dep, method='scaled-dot')
        # Step 4. Compute the loss, gradients, and update the parameters by
        # calling optimizer.step()
        loss.backward()
        optimizer.step()

        train_loss+=loss.item()

    model.eval()
     # 
    _, _, train_acc = accuracy_calculation(model,train_input_index,train_output_index,train_pos_index,train_dep_index)
    _, _, val_acc = accuracy_calculation(model,val_input_index,val_output_index,val_pos_index,val_dep_index)

    val_loss = 0
    for i in range(len(val_input_index)):
        tags_index = val_output_index[i]
        pos_index = val_pos_index[i]
        dep_index = val_dep_index[i]
        sentence_in = torch.tensor(val_input_index[i], dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)
        pos = torch.tensor(pos_index, dtype=torch.long).to(device)
        dep = torch.tensor(dep_index, dtype=torch.long).to(device)
        loss = model.neg_ll(sentence_in, targets, pos, dep, method='scaled-dot')
        val_loss+=loss.item()
    time2 = datetime.datetime.now()

    print("Epoch:%d, Training loss: %.2f, train acc: %.4f, val loss: %.2f, val acc: %.4f, time: %.2fs" %(epoch+1, train_loss,train_acc, val_loss, val_acc, (time2-time1).total_seconds()))

y_pred, y_true, _ = accuracy_calculation(model,val_input_index,val_output_index,val_pos_index,val_dep_index)
def decode_output(output_list):
    ix_to_tag = {v:k for k,v in tag_to_ix.items()}
    return [ix_to_tag[output] for output in output_list]
y_true_decode = decode_output(y_true)
y_pred_decode = decode_output(y_pred)
from sklearn.metrics import classification_report
print(classification_report(y_true_decode,y_pred_decode,digits=4))


Epoch:1, Training loss: 24818.84, train acc: 0.9912, val loss: 1663.70, val acc: 0.9886, time: 611.66s
Epoch:2, Training loss: 3723.22, train acc: 0.9979, val loss: 1371.56, val acc: 0.9946, time: 619.91s
              precision    recall  f1-score   support

           C     0.9707    0.9956    0.9830      1600
           D     0.9296    0.9763    0.9524       379
           O     0.9997    0.9926    0.9961     19120
           P     0.9982    0.9995    0.9989      3931
           S     0.9898    0.9994    0.9946      3290
        SEPA     1.0000    1.0000    1.0000      3603
           T     0.9598    0.9853    0.9724      1431

    accuracy                         0.9945     33354
   macro avg     0.9783    0.9927    0.9853     33354
weighted avg     0.9947    0.9945    0.9946     33354



In [ ]:
import sklearn
from sklearn.metrics import f1_score
sklearn.metrics.f1_score(y_true_decode, y_pred_decode,average='micro')

0.9945433831024765

### Model 2 Without CRF

In [ ]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import math
from numpy.linalg import norm

torch.manual_seed(1)



class BiLSTM_CRF(nn.Module):

    def __init__(self, vocab_size, tag_to_ix, embedding_dim, hidden_dim, wv_embedding_dim, method):
        super(BiLSTM_CRF, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.tag_to_ix = tag_to_ix
        self.tagset_size = len(tag_to_ix)
        self.wv_embedding_dim = wv_embedding_dim
        self.method = method
        self.word_embeds = nn.Embedding(vocab_size, wv_embedding_dim)
        self.word_embeds.weight.data.copy_(torch.from_numpy(embedding_matrix))
        
        self.pos_embeds = nn.Embedding(pos_embedding.shape[0], pos_embedding.shape[0])
        self.pos_embeds.weight.data.copy_(torch.from_numpy(pos_embedding))
        
        self.dep_embeds = nn.Embedding(dep_embedding.shape[0], dep_embedding.shape[0])
        self.dep_embeds.weight.data.copy_(torch.from_numpy(dep_embedding))
        
        self.lstm = nn.LSTM(embedding_dim, hidden_dim // 2,
                            num_layers=1, bidirectional=True,dropout=0.5)

        # Maps the output of the LSTM into tag space.
        self.hidden2tag = nn.Linear(hidden_dim*2, self.tagset_size)


        self.hidden = self.hidden_initial()
        self.dropout_lstm=nn.Dropout(p=0.5)
        
    def hidden_initial(self):
        return (torch.randn(2, 1, self.hidden_dim // 2).to(device), ### different layers 需要改
                torch.randn(2, 1, self.hidden_dim // 2).to(device)) ###


    # method = dot
    def attention(self,lstm_out,hidden_out, hidden_dim, method):

        h_out_T = torch.transpose(hidden_out,1,2)
        
        h_out_T_1 = h_out_T   
        h_out_0 = hidden_out   
        for i in range(lstm_out.size()[0]-1):
          h_out_T = torch.cat((h_out_T, h_out_T_1), 0)
          hidden_out = torch.cat((hidden_out, h_out_0), 0)
        W = nn.Linear(hidden_dim,hidden_dim,bias=False)
        if method == 'dot':
            attention_w = F.softmax(torch.bmm(lstm_out, h_out_T),dim=-1)
        elif method == 'scaled-dot':
            attention_w = F.softmax(torch.bmm(lstm_out, h_out_T)/np.sqrt(self.hidden_dim),dim=-1)
        elif method == 'cosine':
            attention_w = F.softmax(torch.bmm(lstm_out,h_out_T)/(torch.norm(lstm_out)*torch.norm(h_out_T)),dim=-1)

        elif method == 'general':
            attention_w = F.softmax(torch.bmm(lstm_out, W(h_out_T), h_out_T),dim=-1)

        attention_output = torch.bmm(attention_w, hidden_out)
        concat_output = torch.cat((attention_output, lstm_out), 1)
        return concat_output


    def forward(self, sentence, pos, dep, method):
        self.hidden = self.hidden_initial()
        embeds = torch.cat((self.word_embeds(sentence).view(len(sentence), 1, -1),self.pos_embeds(pos).view(len(pos), 1, -1),self.dep_embeds(dep).view(len(dep), 1, -1)),2)
        lstm_out, self.hidden = self.lstm(embeds, self.hidden)
        #print('lstm_out.shape',lstm_out.shape)
        hidden_out = torch.cat((self.hidden[0].view(1, 2, 1, -1)[0,0,:,:],self.hidden[0].view(1, 2, 1, -1)[0,1,:,:]),1) ###################不同layer这里需要改
        hidden_out = hidden_out.unsqueeze(0)
        #print('hidden_out.shape',hidden_out.shape)
        attention_out = self.attention(lstm_out,hidden_out,hidden_dim,method)
        attention_out = attention_out.view(-1,self.hidden_dim*2)
        #print('attention_out.shape',attention_out.shape)
        attention_out = self.dropout_lstm(attention_out)
        lstm_fts = self.hidden2tag(attention_out)
        #print('lstm_fts.shape',lstm_fts.shape)
        return lstm_fts

   

    def neg_ll(self, sentence, tags, pos, dep, method):
        feats = self.lstm_ft_get(sentence, pos, dep, method)
        #print('neg_ll')
        forward_score = self._forward_alg(feats)
        gold_score = self.sentence_scoring(feats, tags)
        return forward_score - gold_score




def accuracy_calculation(model, input_index, output_index, pos_index, dep_index):

  c, total = 0, 0

  pred_list, truth = [], []

  #print('accuracy_calculation')
for i in range(len(input_index)):
    sentence_in = torch.tensor(input_index[i], dtype=torch.long).to(device)
    
    pos = torch.tensor(pos_index[i], dtype=torch.long).to(device)
    dep = torch.tensor(dep_index[i], dtype=torch.long).to(device)



    output = model.forward(sentence_in, pos, dep, 'scaled-dot')
    pred = torch.argmax(output, -1)
    for j in range(len(output_index[i])):
      total += 1
      
      pred_list.append(pred[j])
      truth.append(output_index[i][j])

      if pred[j] == output_index[i][j]:
        c += 1

  accuracy = c/total

  return truth, pred_list, accuracy


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
hidden_dim = 250

model = BiLSTM_CRF(len(full_word_to_ix), tag_to_ix, embedding_dim, hidden_dim, WV_EMBEDDING_DIM, method='scaled-dot').to(device)
optimizer = optim.SGD(model.parameters(), lr=0.1, weight_decay=1e-4)
criterion = nn.CrossEntropyLoss()

import datetime
for epoch in range(2):  
    time1 = datetime.datetime.now()
    train_loss = 0
    method='scaled-dot'
    model.train()
    for i in range(len(train_input_index)):
        #print(i)
        tags_index = train_output_index[i]
        dep_index = train_dep_index[i]
        pos_index = train_pos_index[i]
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is,
        # turn them into Tensors of word indices.
        sentence_in = torch.tensor(train_input_index[i], dtype=torch.long).to(device)
        pos = torch.tensor(pos_index, dtype=torch.long).to(device)
        dep = torch.tensor(dep_index, dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)
        # Step 3. Run our forward pass.
        output = model(sentence_in, pos, dep, 'scaled-dot')

        loss = criterion(output, targets) 
        # Step 4. Compute the loss, gradients, and update the parameters by
        # calling optimizer.step()
        loss.backward()
        optimizer.step()

        train_loss+=loss.item()

    model.eval()
     # 
    _, _, train_acc = accuracy_calculation(model,train_input_index,train_output_index,train_pos_index,train_dep_index)
    _, _, val_acc = accuracy_calculation(model,val_input_index,val_output_index,val_pos_index,val_dep_index)

    val_loss = 0
    for i in range(len(val_input_index)):
        tags_index = val_output_index[i]
        pos_index = val_pos_index[i]
        dep_index = val_dep_index[i]
        sentence_in = torch.tensor(val_input_index[i], dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)
        pos = torch.tensor(pos_index, dtype=torch.long).to(device)
        dep = torch.tensor(dep_index, dtype=torch.long).to(device)
        output = model(sentence_in, pos, dep, 'scaled-dot')
        loss = criterion(output, targets) 
        val_loss+=loss.item()
    time2 = datetime.datetime.now()

    print("Epoch:%d, Training loss: %.2f, train acc: %.4f, val loss: %.2f, val acc: %.4f, time: %.2fs" %(epoch+1, train_loss,train_acc, val_loss, val_acc, (time2-time1).total_seconds()))

y_pred, y_true, _ = accuracy_calculation(model,val_input_index,val_output_index,val_pos_index,val_dep_index)
y_true =  torch.tensor(y_true, device = 'cpu').numpy().tolist()
from sklearn.metrics import classification_report
print(classification_report(y_true, y_pred,digits=4))


Epoch:1, Training loss: 9268.59, train acc: 0.9730, val loss: 964.20, val acc: 0.9730, time: 168.42s
Epoch:2, Training loss: 2539.47, train acc: 0.9899, val loss: 554.88, val acc: 0.9873, time: 160.44s
              precision    recall  f1-score   support

           2     0.9982    0.9868    0.9925     19203
           3     0.9346    0.9628    0.9485      1426
           4     0.9906    0.9990    0.9948      3903
           5     1.0000    1.0000    1.0000      3603
           6     0.9825    0.9942    0.9883      3283
           7     0.7111    0.9248    0.8040       306
           8     0.9494    0.9558    0.9526      1630

    accuracy                         0.9873     33354
   macro avg     0.9381    0.9748    0.9544     33354
weighted avg     0.9882    0.9873    0.9876     33354



In [ ]:
import sklearn
from sklearn.metrics import f1_score
sklearn.metrics.f1_score(y_true, y_pred,average='micro')

0.9872878815134617

## 4.5. Performance Comparison

### Best Model

BiLSTM + CRF + Self Attention(scaled dot-product) + 1 layer <br>
Embedding: Fasttext(training+validation data) + pos + dep + fasttext(dota-wiki + CONDA test)

In [ ]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import math
from numpy.linalg import norm

torch.manual_seed(1)

def argmax(vec):
    # return the argmax as a python int
    _, idx = torch.max(vec, 1)
    return idx.item()


# Compute log sum exp in a numerically stable way for the forward algorithm
def log_sum_exp(vec):
    max_score = vec[0, argmax(vec)]
    max_score_broadcast = max_score.view(1, -1).expand(1, vec.size()[1])
    return max_score + \
        torch.log(torch.sum(torch.exp(vec - max_score_broadcast)))

class BiLSTM_CRF(nn.Module):

    def __init__(self, vocab_size, tag_to_ix, embedding_dim, hidden_dim, wv_embedding_dim, method):
        super(BiLSTM_CRF, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.tag_to_ix = tag_to_ix
        self.tagset_size = len(tag_to_ix)
        self.wv_embedding_dim = wv_embedding_dim
        self.method = method
        self.word_embeds = nn.Embedding(vocab_size, wv_embedding_dim)
        self.word_embeds.weight.data.copy_(torch.from_numpy(embedding_matrix))
        
        self.pos_embeds = nn.Embedding(pos_embedding.shape[0], pos_embedding.shape[0])
        self.pos_embeds.weight.data.copy_(torch.from_numpy(pos_embedding))
        
        self.dep_embeds = nn.Embedding(dep_embedding.shape[0], dep_embedding.shape[0])
        self.dep_embeds.weight.data.copy_(torch.from_numpy(dep_embedding))
        
        self.lstm = nn.LSTM(embedding_dim, hidden_dim // 2,
                            num_layers=1, bidirectional=True,dropout=0.5)

        # Maps the output of the LSTM into tag space.
        self.hidden2tag = nn.Linear(hidden_dim*2, self.tagset_size)

        # Matrix of transition parameters.  Entry i,j is the score of
        # transitioning *to* i *from* j.
        self.transitions = nn.Parameter(
            torch.randn(self.tagset_size, self.tagset_size))

        # These two statements enforce the constraint that we never transfer
        # to the start tag and we never transfer from the stop tag
        self.transitions.data[tag_to_ix[START_TAG], :] = -10000
        self.transitions.data[:, tag_to_ix[STOP_TAG]] = -10000

        self.hidden = self.hidden_initial()
        self.dropout_lstm=nn.Dropout(p=0.5)
        
    def hidden_initial(self):
        return (torch.randn(2, 1, self.hidden_dim // 2).to(device), ### different layers 需要改
                torch.randn(2, 1, self.hidden_dim // 2).to(device)) ###

    def _forward_alg(self, feats):
        # Do the forward algorithm to compute the partition function
        init_alphas = torch.full((1, self.tagset_size), -10000.).to(device)
        # START_TAG has all of the score.
        init_alphas[0][self.tag_to_ix[START_TAG]] = 0.
        #print('_forward_alg')
        # Wrap in a variable so that we will get automatic backprop
        forward_var = init_alphas

        # Iterate through the sentence
        for feat in feats:
            alphas_t = []  # The forward tensors at this timestep
            for next_tag in range(self.tagset_size):
                # broadcast the emission score: it is the same regardless of
                # the previous tag
                emit_score = feat[next_tag].view(
                    1, -1).expand(1, self.tagset_size)
                # the ith entry of trans_score is the score of transitioning to
                # next_tag from i
                trans_score = self.transitions[next_tag].view(1, -1)
                # The ith entry of next_tag_var is the value for the
                # edge (i -> next_tag) before we do log-sum-exp
                next_tag_var = forward_var + trans_score + emit_score
                # The forward variable for this tag is log-sum-exp of all the
                # scores.
                alphas_t.append(log_sum_exp(next_tag_var).view(1))
            forward_var = torch.cat(alphas_t).view(1, -1)
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        alpha = log_sum_exp(terminal_var)
        return alpha
    # method = dot
    def attention(self,lstm_out,hidden_out, hidden_dim, method):

        h_out_T = torch.transpose(hidden_out,1,2)
        
        h_out_T_1 = h_out_T   
        h_out_0 = hidden_out   
        for i in range(lstm_out.size()[0]-1):
          h_out_T = torch.cat((h_out_T, h_out_T_1), 0)
          hidden_out = torch.cat((hidden_out, h_out_0), 0)
        W = nn.Linear(hidden_dim,hidden_dim,bias=False)
        if method == 'dot':
            attention_w = F.softmax(torch.bmm(lstm_out, h_out_T),dim=-1)
        elif method == 'scaled-dot':
            attention_w = F.softmax(torch.bmm(lstm_out, h_out_T)/np.sqrt(self.hidden_dim),dim=-1)
        elif method == 'cosine':
            attention_w = F.softmax(torch.bmm(lstm_out,h_out_T)/(torch.norm(lstm_out)*torch.norm(h_out_T)),dim=-1)

        elif method == 'general':
            attention_w = F.softmax(torch.bmm(lstm_out, W(h_out_T), h_out_T),dim=-1)

        attention_output = torch.bmm(attention_w, hidden_out)
        concat_output = torch.cat((attention_output, lstm_out), 1)
        return concat_output

    def lstm_ft_get(self, sentence, pos, dep, method):
        self.hidden = self.hidden_initial()
        embeds = torch.cat((self.word_embeds(sentence).view(len(sentence), 1, -1),self.pos_embeds(pos).view(len(pos), 1, -1),self.dep_embeds(dep).view(len(dep), 1, -1)),2)
        lstm_out, self.hidden = self.lstm(embeds, self.hidden)
        #print('lstm_out.shape',lstm_out.shape)
        hidden_out = torch.cat((self.hidden[0].view(1, 2, 1, -1)[0,0,:,:],self.hidden[0].view(1, 2, 1, -1)[0,1,:,:]),1) ###################不同layer这里需要改
        hidden_out = hidden_out.unsqueeze(0)
        #print('hidden_out.shape',hidden_out.shape)
        attention_out = self.attention(lstm_out,hidden_out,hidden_dim,method)
        attention_out = attention_out.view(-1,self.hidden_dim*2)
        #print('attention_out.shape',attention_out.shape)
        attention_out = self.dropout_lstm(attention_out)
        lstm_fts = self.hidden2tag(attention_out)
        #print('lstm_fts.shape',lstm_fts.shape)
        return lstm_fts

    def sentence_scoring(self, feats, tags):
        # Gives the score of a provided tag sequence
        score = torch.zeros(1).to(device)
        tags = torch.cat([torch.tensor([self.tag_to_ix[START_TAG]], dtype=torch.long).to(device), tags])
        for i, feat in enumerate(feats):
            score = score + \
                self.transitions[tags[i + 1], tags[i]] + feat[tags[i + 1]]
        score = score + self.transitions[self.tag_to_ix[STOP_TAG], tags[-1]]
        return score

    def viterbi_decoding(self, feats):
        backpointers = []

        # Initialize the viterbi variables in log space
        init_vvars = torch.full((1, self.tagset_size), -10000.).to(device)
        init_vvars[0][self.tag_to_ix[START_TAG]] = 0

        # forward_var at step i holds the viterbi variables for step i-1
        forward_var = init_vvars
        for feat in feats:
            bptrs_t = []  # holds the backpointers for this step
            viterbivars_t = []  # holds the viterbi variables for this step

            for next_tag in range(self.tagset_size):
                # next_tag_var[i] holds the viterbi variable for tag i at the
                # previous step, plus the score of transitioning
                # from tag i to next_tag.
                # We don't include the emission scores here because the max
                # does not depend on them (we add them in below)
                next_tag_var = forward_var + self.transitions[next_tag]
                best_tag_id = argmax(next_tag_var)
                bptrs_t.append(best_tag_id)
                viterbivars_t.append(next_tag_var[0][best_tag_id].view(1))
            # Now add in the emission scores, and assign forward_var to the set
            # of viterbi variables we just computed
            forward_var = (torch.cat(viterbivars_t) + feat).view(1, -1)
            backpointers.append(bptrs_t)

        # Transition to STOP_TAG
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        best_tag_id = argmax(terminal_var)
        path_score = terminal_var[0][best_tag_id]

        # Follow the back pointers to decode the best path.
        best_path = [best_tag_id]
        for bptrs_t in reversed(backpointers):
            best_tag_id = bptrs_t[best_tag_id]
            best_path.append(best_tag_id)
        # Pop off the start tag (we dont want to return that to the caller)
        start = best_path.pop()
        assert start == self.tag_to_ix[START_TAG]  # Sanity check
        best_path.reverse()
        return path_score, best_path

    def neg_ll(self, sentence, tags, pos, dep, method):
        feats = self.lstm_ft_get(sentence, pos, dep, method)
        #print('neg_ll')
        forward_score = self._forward_alg(feats)
        gold_score = self.sentence_scoring(feats, tags)
        return forward_score - gold_score

    #def forward(self, sentence, ent, pos, dep, bert):  # dont confuse this with _forward_alg above.
    def forward(self, sentence, pos, dep):
        #print('forward')
        # Get the emission scores from the BiLSTM

        lstm_fts = self.lstm_ft_get(sentence, pos, dep, method)
        # Find the best path, given the features.
        score, tag_seq = self.viterbi_decoding(lstm_fts)
        return score, tag_seq



def accuracy_calculation(model, input_index, output_index, pos_index, dep_index):

  c, total = 0, 0

  pred_list, truth = [], []

  #print('accuracy_calculation')
for i in range(len(input_index)):
    sentence_in = torch.tensor(idxs, dtype=torch.long).to(device)
    
    pos = torch.tensor(pos_index[i], dtype=torch.long).to(device)
    dep = torch.tensor(dep_index[i], dtype=torch.long).to(device)



    _, pred = model.forward(sentence_in, pos, dep)
    for j in range(len(output_index[i])):
      total += 1
      
      pred_list.append(pred[j])
      truth.append(output_index[i][j])

      if pred[j] == output_index[i][j]:
        c += 1

  accuracy = c/total

  return truth, pred_list, accuracy


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
hidden_dim = 250

model = BiLSTM_CRF(len(full_word_to_ix), tag_to_ix, embedding_dim, hidden_dim, WV_EMBEDDING_DIM, method='scaled-dot').to(device)
optimizer = optim.SGD(model.parameters(), lr=0.1, weight_decay=1e-4)

import datetime
for epoch in range(2):  
    time1 = datetime.datetime.now()
    train_loss = 0
    method='scaled-dot'
    model.train()
    for i in range(len(train_input_index)):
        #print(i)
        tags_index = train_output_index[i]
        dep_index = train_dep_index[i]
        pos_index = train_pos_index[i]
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is,
        # turn them into Tensors of word indices.
        sentence_in = torch.tensor(train_input_index[i], dtype=torch.long).to(device)
        pos = torch.tensor(pos_index, dtype=torch.long).to(device)
        dep = torch.tensor(dep_index, dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)
        # Step 3. Run our forward pass.
        loss = model.neg_ll(sentence_in, targets, pos, dep, method='scaled-dot')
        # Step 4. Compute the loss, gradients, and update the parameters by
        # calling optimizer.step()
        loss.backward()
        optimizer.step()

        train_loss+=loss.item()

    model.eval()
     # 
    _, _, train_acc = accuracy_calculation(model,train_input_index,train_output_index,train_pos_index,train_dep_index)
    _, _, val_acc = accuracy_calculation(model,val_input_index,val_output_index,val_pos_index,val_dep_index)

    val_loss = 0
    for i in range(len(val_input_index)):
        tags_index = val_output_index[i]
        pos_index = val_pos_index[i]
        dep_index = val_dep_index[i]
        sentence_in = torch.tensor(val_input_index[i], dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)
        pos = torch.tensor(pos_index, dtype=torch.long).to(device)
        dep = torch.tensor(dep_index, dtype=torch.long).to(device)
        loss = model.neg_ll(sentence_in, targets, pos, dep, method='scaled-dot')
        val_loss+=loss.item()
    time2 = datetime.datetime.now()

    print("Epoch:%d, Training loss: %.2f, train acc: %.4f, val loss: %.2f, val acc: %.4f, time: %.2fs" %(epoch+1, train_loss,train_acc, val_loss, val_acc, (time2-time1).total_seconds()))

y_pred, y_true, _ = accuracy_calculation(model,val_input_index,val_output_index,val_pos_index,val_dep_index)
def decode_output(output_list):
    ix_to_tag = {v:k for k,v in tag_to_ix.items()}
    return [ix_to_tag[output] for output in output_list]
y_true_decode = decode_output(y_true)
y_pred_decode = decode_output(y_pred)
from sklearn.metrics import classification_report
print(classification_report(y_true_decode,y_pred_decode,digits=4))


Epoch:1, Training loss: 24818.84, train acc: 0.9912, val loss: 1663.70, val acc: 0.9886, time: 611.66s
Epoch:2, Training loss: 3723.22, train acc: 0.9979, val loss: 1371.56, val acc: 0.9946, time: 619.91s
              precision    recall  f1-score   support

           C     0.9707    0.9956    0.9830      1600
           D     0.9296    0.9763    0.9524       379
           O     0.9997    0.9926    0.9961     19120
           P     0.9982    0.9995    0.9989      3931
           S     0.9898    0.9994    0.9946      3290
        SEPA     1.0000    1.0000    1.0000      3603
           T     0.9598    0.9853    0.9724      1431

    accuracy                         0.9945     33354
   macro avg     0.9783    0.9927    0.9853     33354
weighted avg     0.9947    0.9945    0.9946     33354



In [ ]:
import sklearn
from sklearn.metrics import f1_score
sklearn.metrics.f1_score(y_true_decode, y_pred_decode,average='micro')

0.9945433831024765

### Baseline Model

BiLSTM + CRF  + 1 layer <br>
Embedding: Fasttext(training+validation data) + pos + dep + fasttext(dota-wiki + CONDA test)

In [ ]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import math
from numpy.linalg import norm

torch.manual_seed(1)

def argmax(vec):
    # return the argmax as a python int
    _, idx = torch.max(vec, 1)
    return idx.item()


# Compute log sum exp in a numerically stable way for the forward algorithm
def log_sum_exp(vec):
    max_score = vec[0, argmax(vec)]
    max_score_broadcast = max_score.view(1, -1).expand(1, vec.size()[1])
    return max_score + \
        torch.log(torch.sum(torch.exp(vec - max_score_broadcast)))

class BiLSTM_CRF(nn.Module):

    def __init__(self, vocab_size, tag_to_ix, embedding_dim, hidden_dim, wv_embedding_dim):
        super(BiLSTM_CRF, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.tag_to_ix = tag_to_ix
        self.tagset_size = len(tag_to_ix)
        self.wv_embedding_dim = wv_embedding_dim
        self.word_embeds = nn.Embedding(vocab_size, wv_embedding_dim)
        self.word_embeds.weight.data.copy_(torch.from_numpy(embedding_matrix))
        
        self.pos_embeds = nn.Embedding(pos_embedding.shape[0], pos_embedding.shape[0])
        self.pos_embeds.weight.data.copy_(torch.from_numpy(pos_embedding))
        
        self.dep_embeds = nn.Embedding(dep_embedding.shape[0], dep_embedding.shape[0])
        self.dep_embeds.weight.data.copy_(torch.from_numpy(dep_embedding))
        
        self.lstm = nn.LSTM(embedding_dim, hidden_dim // 2,
                            num_layers=1, bidirectional=True,dropout=0.5)

        # Maps the output of the LSTM into tag space.
        self.hidden2tag = nn.Linear(hidden_dim, self.tagset_size)

        # Matrix of transition parameters.  Entry i,j is the score of
        # transitioning *to* i *from* j.
        self.transitions = nn.Parameter(
            torch.randn(self.tagset_size, self.tagset_size))

        # These two statements enforce the constraint that we never transfer
        # to the start tag and we never transfer from the stop tag
        self.transitions.data[tag_to_ix[START_TAG], :] = -10000
        self.transitions.data[:, tag_to_ix[STOP_TAG]] = -10000

        self.hidden = self.hidden_initial()
        self.dropout_lstm=nn.Dropout(p=0.5)
        
    def hidden_initial(self):
        return (torch.randn(2, 1, self.hidden_dim // 2).to(device),
                torch.randn(2, 1, self.hidden_dim // 2).to(device))

    def _forward_alg(self, feats):
        # Do the forward algorithm to compute the partition function
        init_alphas = torch.full((1, self.tagset_size), -10000.).to(device)
        # START_TAG has all of the score.
        init_alphas[0][self.tag_to_ix[START_TAG]] = 0.
        #print('_forward_alg')
        # Wrap in a variable so that we will get automatic backprop
        forward_var = init_alphas

        # Iterate through the sentence
        for feat in feats:
            alphas_t = []  # The forward tensors at this timestep
            for next_tag in range(self.tagset_size):
                # broadcast the emission score: it is the same regardless of
                # the previous tag
                emit_score = feat[next_tag].view(
                    1, -1).expand(1, self.tagset_size)
                # the ith entry of trans_score is the score of transitioning to
                # next_tag from i
                trans_score = self.transitions[next_tag].view(1, -1)
                # The ith entry of next_tag_var is the value for the
                # edge (i -> next_tag) before we do log-sum-exp
                next_tag_var = forward_var + trans_score + emit_score
                # The forward variable for this tag is log-sum-exp of all the
                # scores.
                alphas_t.append(log_sum_exp(next_tag_var).view(1))
            forward_var = torch.cat(alphas_t).view(1, -1)
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        alpha = log_sum_exp(terminal_var)
        return alpha
    # method = dot

    def lstm_ft_get(self, sentence, pos, dep):
        self.hidden = self.hidden_initial()
        embeds = torch.cat((self.word_embeds(sentence).view(len(sentence), 1, -1),self.pos_embeds(pos).view(len(pos), 1, -1),self.dep_embeds(dep).view(len(dep), 1, -1)),2)
        lstm_out, self.hidden = self.lstm(embeds, self.hidden)
        lstm_out = lstm_out.view(len(sentence), self.hidden_dim)
        lstm_fts = self.hidden2tag(lstm_out)
        return lstm_fts


    def sentence_scoring(self, feats, tags):
        # Gives the score of a provided tag sequence
        score = torch.zeros(1).to(device)
        tags = torch.cat([torch.tensor([self.tag_to_ix[START_TAG]], dtype=torch.long).to(device), tags])
        for i, feat in enumerate(feats):
            score = score + \
                self.transitions[tags[i + 1], tags[i]] + feat[tags[i + 1]]
        score = score + self.transitions[self.tag_to_ix[STOP_TAG], tags[-1]]
        return score

    def viterbi_decoding(self, feats):
        backpointers = []

        # Initialize the viterbi variables in log space
        init_vvars = torch.full((1, self.tagset_size), -10000.).to(device)
        init_vvars[0][self.tag_to_ix[START_TAG]] = 0

        # forward_var at step i holds the viterbi variables for step i-1
        forward_var = init_vvars
        for feat in feats:
            bptrs_t = []  # holds the backpointers for this step
            viterbivars_t = []  # holds the viterbi variables for this step

            for next_tag in range(self.tagset_size):
                # next_tag_var[i] holds the viterbi variable for tag i at the
                # previous step, plus the score of transitioning
                # from tag i to next_tag.
                # We don't include the emission scores here because the max
                # does not depend on them (we add them in below)
                next_tag_var = forward_var + self.transitions[next_tag]
                best_tag_id = argmax(next_tag_var)
                bptrs_t.append(best_tag_id)
                viterbivars_t.append(next_tag_var[0][best_tag_id].view(1))
            # Now add in the emission scores, and assign forward_var to the set
            # of viterbi variables we just computed
            forward_var = (torch.cat(viterbivars_t) + feat).view(1, -1)
            backpointers.append(bptrs_t)

        # Transition to STOP_TAG
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        best_tag_id = argmax(terminal_var)
        path_score = terminal_var[0][best_tag_id]

        # Follow the back pointers to decode the best path.
        best_path = [best_tag_id]
        for bptrs_t in reversed(backpointers):
            best_tag_id = bptrs_t[best_tag_id]
            best_path.append(best_tag_id)
        # Pop off the start tag (we dont want to return that to the caller)
        start = best_path.pop()
        assert start == self.tag_to_ix[START_TAG]  # Sanity check
        best_path.reverse()
        return path_score, best_path

    def neg_ll(self, sentence, tags, pos, dep):
        feats = self.lstm_ft_get(sentence, pos, dep)
        forward_score = self._forward_alg(feats)
        gold_score = self.sentence_scoring(feats, tags)
        return forward_score - gold_score

    #def forward(self, sentence, ent, pos, dep, bert):  # dont confuse this with _forward_alg above.
    def forward(self, sentence, pos, dep):
        #print('forward')
        # Get the emission scores from the BiLSTM

        lstm_fts = self.lstm_ft_get(sentence, pos, dep)
        # Find the best path, given the features.
        score, tag_seq = self.viterbi_decoding(lstm_fts)
        return score, tag_seq



def accuracy_calculation(model, input_index, output_index, pos_index, dep_index):

  c, total = 0, 0

  pred_list, truth = [], []

  #print('accuracy_calculation')
  for i in range(len(input_index)):
    sentence_in = torch.tensor(input_index[i], dtype=torch.long).to(device)
    
    pos = torch.tensor(pos_index[i], dtype=torch.long).to(device)
    dep = torch.tensor(dep_index[i], dtype=torch.long).to(device)



    _, pred = model.forward(sentence_in, pos, dep)
    for j in range(len(output_index[i])):
      total += 1
      
      pred_list.append(pred[j])
      truth.append(output_index[i][j])

      if pred[j] == output_index[i][j]:
        c += 1

  accuracy = c/total

  return truth, pred_list, accuracy

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
hidden_dim = 250

model = BiLSTM_CRF(len(full_word_to_ix), tag_to_ix, embedding_dim, hidden_dim, WV_EMBEDDING_DIM).to(device)
optimizer = optim.SGD(model.parameters(), lr=0.12, weight_decay=1e-4)

import datetime
for epoch in range(2):  
    time1 = datetime.datetime.now()
    train_loss = 0
    model.train()
    for i in range(len(train_input_index)):
        #print(i)
        tags_index = train_output_index[i]
        dep_index = train_dep_index[i]
        pos_index = train_pos_index[i]
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is,
        # turn them into Tensors of word indices.
        sentence_in = torch.tensor(train_input_index[i], dtype=torch.long).to(device)
        pos = torch.tensor(pos_index, dtype=torch.long).to(device)
        dep = torch.tensor(dep_index, dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)
        # Step 3. Run our forward pass.
        loss = model.neg_ll(sentence_in, targets, pos, dep)
        # Step 4. Compute the loss, gradients, and update the parameters by
        # calling optimizer.step()
        loss.backward()
        optimizer.step()

        train_loss+=loss.item()

    model.eval()
     # 
    _, _, train_acc = accuracy_calculation(model,train_input_index,train_output_index,train_pos_index,train_dep_index)
    _, _, val_acc = accuracy_calculation(model,val_input_index,val_output_index,val_pos_index,val_dep_index)

    val_loss = 0
    for i in range(len(val_input_index)):
        tags_index = val_output_index[i]
        pos_index = val_pos_index[i]
        dep_index = val_dep_index[i]
        sentence_in = torch.tensor(val_input_index[i], dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)
        pos = torch.tensor(pos_index, dtype=torch.long).to(device)
        dep = torch.tensor(dep_index, dtype=torch.long).to(device)
        loss = model.neg_ll(sentence_in, targets, pos, dep)
        val_loss+=loss.item()
    time2 = datetime.datetime.now()

    print("Epoch:%d, Training loss: %.2f, train acc: %.4f, val loss: %.2f, val acc: %.4f, time: %.2fs" %(epoch+1, train_loss,train_acc, val_loss, val_acc, (time2-time1).total_seconds()))

y_pred, y_true, _ = accuracy_calculation(model,val_input_index,val_output_index,val_pos_index,val_dep_index)
def decode_output(output_list):
    ix_to_tag = {v:k for k,v in tag_to_ix.items()}
    return [ix_to_tag[output] for output in output_list]
y_true_decode = decode_output(y_true)
y_pred_decode = decode_output(y_pred)
from sklearn.metrics import classification_report
print(classification_report(y_true_decode,y_pred_decode,digits=4))


Epoch:1, Training loss: 13779.81, train acc: 0.9962, val loss: 1135.65, val acc: 0.9933, time: 571.77s
Epoch:2, Training loss: 1452.23, train acc: 0.9989, val loss: 1138.28, val acc: 0.9952, time: 558.08s
              precision    recall  f1-score   support

           C     0.9726    0.9981    0.9852      1599
           D     0.9497    0.9895    0.9692       382
           O     0.9997    0.9931    0.9964     19111
           P     0.9987    0.9997    0.9992      3932
           S     0.9928    0.9985    0.9956      3303
        SEPA     1.0000    1.0000    1.0000      3603
           T     0.9639    0.9944    0.9789      1424

    accuracy                         0.9954     33354
   macro avg     0.9825    0.9962    0.9892     33354
weighted avg     0.9956    0.9954    0.9955     33354



In [ ]:
import sklearn
from sklearn.metrics import f1_score
sklearn.metrics.f1_score(y_true_decode, y_pred_decode,average='micro')

0.9954428254482222